# Init

In [ ]:
from IPython.display import clear_output
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

CACHE = {}
class SkipExecution(Exception):
    def _render_traceback_(self):
        print("Skip execution")
        clear_output()

try:
    import python_speech_features
except:
    pass
else:
    raise SkipExecution

!pip install kaggle python_speech_features dtw fastdtw dtaidistance AudAugio pyrubberband --upgrade -q
!apt install soundstretch rubberband-cli librubberband2 libsndfile1 > /dev/null
# !kaggle datasets download -d mfekadu/darpa-timit-acousticphonetic-continuous-speech > /dev/null
!gdown -O data.zip --id "15MxBckNzyEjO7cpY38O38NaWnssShl2l"
!unzip data.zip > /dev/null
!git clone https://github.com/vegetablejuiceftw/soft-pointer-networks spn
# !mv soft-pointer-networks/* .

clear_output()

In [ ]:
# packges
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import IPython.display as ipd
from python_speech_features import logfbank, mfcc
from matplotlib import cm
from matplotlib.pyplot import figure
from collections import Counter
import json
from time import sleep
from google.colab import drive
from numpy import dot
from numpy.linalg import norm
import cProfile
from typing import NamedTuple, List
from random import choice, sample, Random
import random
import math
import contextlib
from sklearn.model_selection import train_test_split

import soundfile as sf
import pyrubberband as pyrb
import scipy.io.wavfile as wavfile
import librosa
from IPython.display import Audio, Image, display

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from collections import namedtuple
from torchtext.legacy.data import BucketIterator, Field

import numpy as np
from os.path import join
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch.nn as nn
import sys
from matplotlib.pyplot import figure
import json
import gc
import uuid

from pprint import pprint
from collections import OrderedDict, defaultdict
from itertools import chain

from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import BucketIterator, RawField
from collections import namedtuple
from torchtext.legacy.data import BucketIterator, Field

from dtw import dtw, accelerated_dtw as adtw
from fastdtw import fastdtw, dtw as slowdtw
from dtaidistance import dtw as dtaidtw
from dtaidistance.dtw_ndim import warping_paths

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# caching
duration_model = None
duration_cache = {}

BASE_PATH = '/content/TIMIT-PLUS'
FULL_FODLER_PATH = join(BASE_PATH, 'data')
INPUT_SIZE = 26

# WIN_STEP = 0.015
WIN_STEP = 0.010
# WIN_STEP = 0.005

WIN_SIZE = 0.025
# WIN_SIZE = 0.030
# WIN_SIZE = 0.020
# WIN_SIZE = 0.010
# WIN_SIZE = 0.015
# WIN_SIZE = 0.020

DURATION_SCALER = 256.  # duration models do not play well with 123ms bot not also with 0.1sec ...

FOUND_LABELS = dict([
  ('h#', 12600), ('ix', 11587), ('s', 10114), ('iy', 9663), ('n', 9569), ('r', 9064),
  ('tcl', 8978), ('l', 8157), ('kcl', 7823), ('ih', 6760), ('dcl', 6585), ('k', 6488),
  ('t', 5899), ('m', 5429), ('ae', 5404), ('eh', 5293), ('z', 5046), ('ax', 4956), ('q', 4834),
  ('d', 4793), ('axr', 4790), ('w', 4379), ('aa', 4197), ('ao', 4096), ('dh', 3879),
  ('dx', 3649), ('pcl', 3609), ('p', 3545), ('ay', 3242), ('ah', 3185), ('f', 3128),
  ('ey', 3088), ('b', 3067), ('sh', 3034), ('gcl', 3031), ('ow', 2913), ('er', 2846),
  ('g', 2772), ('v', 2704), ('bcl', 2685), ('ux', 2488), ('y', 2349), ('epi', 2000),
  ('ng', 1744), ('jh', 1581), ('hv', 1523), ('pau', 1343), ('nx', 1331), ('hh', 1313),
  ('el', 1294), ('ch', 1081), ('th', 1018), ('en', 974), ('oy', 947), ('aw', 945),
  ('uh', 756), ('uw', 725), ('ax-h', 493), ('zh', 225), ('em', 171), ('eng', 43), ])

TRANSFORM_MAPPING = {
    # First, the sentence-beginning and sentence-ending pause symbols /h#/ were mapped to pause (/pau/).
    'h#': 'pau',
    # Epenthetic silence (/epi/) was also mapped to pause.
    'epi': 'pau',
    # The syllabic phonemes /em/, /en/, /eng/, and /el/ were mapped to their non-syllabic counterparts /m/, /n/, /ng/, and /l/, respectively.
    "em" : "m",
    "en" : "n",
    "eng" : "ng",
    "el" : "l",
    # The glottal closure symbol /q/ was merged based on weird rules
    "q": None,
}

NO_BORDER_MAPPING = {
    'pau', 'pcl', 'bcl', 'tcl', 'dcl', 'kcl', 'gcl',
}


KNOWN_LABELS = list(sorted(set(TRANSFORM_MAPPING.get(k, k) for k in sorted(FOUND_LABELS.keys()) if TRANSFORM_MAPPING.get(k, k))))
KNOWN_LABELS_COUNT = len(KNOWN_LABELS)
MAP_LABELS = {
    label: (
        [int(KNOWN_LABELS.index(label) == i) for i in range(KNOWN_LABELS_COUNT)],
        KNOWN_LABELS.index(label),
    )
    for label in KNOWN_LABELS
}

class AudioCaching:
    BASE_FOLDER = '/content/TIMIT-PLUS'
    CACHE_FOLDER = BASE_FOLDER + '-CACHE'
    # AUDIO_SCALING = 32768.
    AUDIO_RATE = 16000

    @classmethod
    def to_file_path(cls, file_path, key):
        filename = os.path.basename(file_path)
        cache_filename = f"[{'__'.join(str(k) for k in key)}]{filename}"
        cache_path = os.path.dirname(file_path).replace(cls.BASE_FOLDER, cls.CACHE_FOLDER)
        cache_file_path = os.path.join(cache_path, cache_filename)
        return cache_file_path

    @classmethod
    def set(cls, file_path, key, audio):
        cache_file_path = cls.to_file_path(file_path, key)
        # cached version exists :D
        if not os.path.isfile(cache_file_path):
            os.makedirs(os.path.dirname(cache_file_path), exist_ok=True)
            sf.write(cache_file_path, audio, cls.AUDIO_RATE)

    @classmethod
    def get(cls, file_path, key):
        cache_file_path = cls.to_file_path(file_path, key)
        # cached version exists :D
        if os.path.isfile(cache_file_path):
            # loader = wavfile.read(cache_file_path)[1]
            # audio = audio.astype(np.float32) / audio_scaling
            print("+", cache_file_path)
            return cls.load(cache_file_path)
        else:
            print("-", file_path)

    @classmethod
    def load(cls, file_path):
        audio = sf.read(file_path)[0]
        return audio

class UtteranceBatch(NamedTuple):
    padded: torch.tensor
    masks: torch.tensor
    lengths: torch.tensor

class Utterance(NamedTuple):
    features: torch.tensor
    labels: torch.tensor
    transcription: torch.tensor
    label_vec: torch.tensor
    out_map: List
    out_duration: torch.tensor
    in_transcription: torch.tensor
    position: torch.tensor
    border: torch.tensor
    weight: torch.tensor

    index: int
    key: str
    audio_file: str
    label_file: str

def stack(arr, tensor):
    return [tensor(a).to(device) for a in arr]

def dedupe(tags):
    labels = []
    last = None
    for x in tags:
        if last is not None and last == x:
            continue
        last = x
        labels.append(x)
    return labels

def find_borders(output_ids, original_mapping, ms_per_step=int(WIN_STEP * 1000)):
    # add half to the border, as it is between the two frames
    arr = (np.where(output_ids[:-1] != output_ids[1:])[0] + 0.5) * ms_per_step
    last = output_ids.shape[0] * ms_per_step
    arr = np.append(arr, last)
    flat_ids = np.array(dedupe(output_ids))
    ids = np.array([voc for voc, end in original_mapping])
    assert (ids.shape == flat_ids.shape) and (ids != flat_ids).sum() == 0, f"[error] Mapping and Output have same composition {ids.shape} {flat_ids.shape}"
    a, b = np.array([end for voc, end in original_mapping]), arr

    diff = (a - b)
    return a, b, diff

class PositionalEncodingLabeler(nn.Module):
    def __init__(self, d_model, dropout=0.1, scale=1, max_len=2048):
        super().__init__()
        self.scale = scale
        self.dropout = nn.Dropout(p=dropout)
        if not self.scale:
            return
        max_len = int(max_len * scale)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        m = nn.Upsample(scale_factor=(1. / scale, 1), mode='bilinear', align_corners=True)
        shape = pe.shape
        pe = pe.view(1, 1, *shape)
        pe = m(pe).view(-1, d_model)

        pe = pe.unsqueeze(0).transpose(0, 1).transpose(0, 1)[0]
        self.register_buffer('pe', pe)

    def forward(self, out_duration):
        durations = torch.cumsum(out_duration, dim=0) * DURATION_SCALER / ms_per_step
        return self.pe[durations.round().long()], durations

ms_per_step = WIN_STEP * 1000
POS_DIM = 32
POS_SCALE = 1
pos_prep = PositionalEncodingLabeler(POS_DIM, scale=POS_SCALE)
MERGE_DOUBLES = False
unholy = set()

class DirectMaskDataset(Dataset):
    base = '/content'

    @classmethod
    def load_csv(cls, prefix, sa=False):
        file_path = join(cls.base, f'{prefix}_data.csv')
        """ Filters out the files names of phonetic and sound data in pairs"""
        df = pd.read_csv(file_path, delimiter=',', nrows = None)
        df = df.sort_values(by=['path_from_data_dir'])
        # audio_mask = df.is_converted_audio == True
        audio_mask = (df.is_audio == True) & (df.is_converted_audio == True)
        phn_mask = df.filename.str.contains('.PHN') == True
        SA_mask = df.filename.str.contains('SA') == False
        df = df.loc[audio_mask | phn_mask]
        print("SA", sa)
        if not sa:
            df = df.loc[SA_mask]

        ipd.display(df.head())
        nRow, nCol = df.shape
        # print(f'There are {nRow} rows and {nCol} columns')
        A, B = df.loc[phn_mask].path_from_data_dir, df.loc[audio_mask].path_from_data_dir
        assert len(A) == len(B)
        files = list(zip(A, B))
        return files

    @staticmethod
    def get_name(file_name):
        _, a, b, name = file_name.split('/')
        name = name.split('.')[0]
        return a, b, name

    def json_to_vec(arr):
        return [np.array([MAP_LABELS[tag][0] for tag in tags]) for tags in arr]

    def process_audio(self, labels: list, length: int, step: float) -> List[List]:
        tag_ints, tag_vecs, tag_mapping, transcription = [], [], [], []

        current, prev = 0, None
        for end_ms, tag in labels:

            tag = TRANSFORM_MAPPING.get(tag, tag)
            q_tag = False # /q/ tag
            if tag is None:
                tag = prev
                q_tag = True
                end_ms = (current + end_ms) / 2

            if current >= end_ms:
                continue

            unholy_combination = prev in NO_BORDER_MAPPING and tag in NO_BORDER_MAPPING
            if unholy_combination:
                unholy.add((prev, tag))

            if prev == tag and MERGE_DOUBLES or unholy_combination or q_tag:
                tag_id, ems = tag_mapping[-1]
                tag_mapping[-1] = (tag_id, end_ms)

            else:
                tag_id = MAP_LABELS[tag][1]
                tag_vec = np.array(MAP_LABELS[tag][0])

                tag_mapping.append((tag_id, end_ms))
                transcription.append(tag_vec)

            prev = tag  # handle same tag occurence

            tag_ints.append(tag_id)
            tag_vecs.append(tag_vec)
            current += step

            while current < end_ms and len(tag_ints) < length:
                tag_ints.append(tag_id)
                tag_vecs.append(tag_vec)
                current += step

        if length > len(tag_vecs):
            tag_ints.append(tag_id)
            tag_vecs.append(tag_vec)

        return tag_ints, tag_vecs, tag_mapping, transcription

    def get_set(self, key, func):
        value = CACHE.get(key)
        value = value if value is not None else func()
        CACHE[key] = value
        return value

    def __init__(self, files, limit=None, mask=None, augment=False, duplicate=1, seed="42"):

        random = Random(seed)  # init random generator
        self.counts = []

        base = self.base

        if limit is not None:
            files = files[:limit]

        inp, out_vec, out_int, out_map, out_dur, out_trans = [], [], [], [], [], []
        position, border, weight = [], [], []

        duplicate_set = set()
        self.files = []

        for i, (label_file, audio_file) in enumerate(files * duplicate):
            assert self.get_name(label_file) == self.get_name(audio_file)
            a, b, c = self.get_name(label_file)
            identifier = f'{a}_{b}_{c}_{i}'

            label_file = os.path.join(base, 'data', label_file)
            audio_file = os.path.join(base, 'data', audio_file)

            loader = lambda: AudioCaching.load(audio_file)
            audio = self.get_set(audio_file, loader)
            audio_scaling, rate = 32768. / 512, 16000
            audio_base_len = len(audio)

            stretch = 1
            pure_key = (audio_file, "pure_key")
            if pure_key not in duplicate_set:
                duplicate_set.add(pure_key)
            elif augment:
                # pitch = random.choice([-6, -4, -1, 1, 4, 6])
                # stretch = random.choice([0.85, 0.9, 0.95, 1.05, 1.1, 1.15])
                # pitch = random.choice([-4, -1, 1, 4])
                pitch = random.choice([-1, 0, 1])
                stretch = random.choice([0.9, 0.95, 1.05, 1.1])

                key_stretch = "time_stretch", stretch
                key_pitch = "pitch_shift", pitch, stretch

                duplication_key = (audio_file, key_pitch)
                if duplication_key in duplicate_set:
                    continue
                duplicate_set.add(duplication_key)

                audio_pitch_shift = lambda: pyrb.pitch_shift(audio, rate, pitch)

                cache_audio = AudioCaching.get(audio_file, key_pitch)
                got_pitch = cache_audio is not None
                cache_audio = cache_audio if got_pitch else AudioCaching.get(audio_file, key_stretch)
                got_stretch = cache_audio is not None

                audio = cache_audio if cache_audio is not None else audio

                if not got_stretch:
                    audio = pyrb.time_stretch(audio, rate, stretch)
                    AudioCaching.set(audio_file, key_stretch, audio)

                if not got_pitch:
                    audio = pyrb.pitch_shift(audio, rate, pitch)
                    AudioCaching.set(audio_file, key_pitch, audio)

                stretch =  len(audio) / audio_base_len

            fbank_feat = logfbank(audio, rate, winlen=WIN_SIZE, winstep=WIN_STEP, nfilt=INPUT_SIZE)  # TODO: remove scaling
            # some audio instances are too short for the audio transcription and the winlen cut :(
            fbank_feat = np.vstack([fbank_feat] + [fbank_feat[-1]] * 10)

            step_size = (WIN_STEP * 1000)
            with open(label_file) as f:
                lines = list(f.readlines())
                length = fbank_feat.shape[0]
                length_ms = length * step_size
                labels = []
                ms_samples = 16

                for line in lines:
                    _, end, tag = line.split()
                    end_ms = float(end) / ms_samples * stretch
                    end_ms = min(end_ms, length_ms)
                    labels.append((end_ms, tag))

                length = int((end_ms / step_size))

            tag_ints, tag_vecs, tag_mapping, transcription = self.process_audio(labels, length, step_size)
            fbank_feat = fbank_feat[:len(tag_ints)]
            length = fbank_feat.shape[0]
            length_ms = length * step_size

            w = [200. / FOUND_LABELS[KNOWN_LABELS[_pid]] for _pid, _ms in tag_mapping]

            if i % 150 == 0:
                print(i)
                gc.collect()

            if length == len(tag_vecs) and length == len(tag_ints):
                original = stack([tag_vecs], torch.FloatTensor)[0].cpu().numpy()
                original_ids = np.argmax(original, axis=1)
                if MERGE_DOUBLES:
                    a, b, diff = find_borders(original_ids, tag_mapping)
                    d = abs(diff).max()
                    if d > 15:
                        print(f"[DIFF-ERROR] diff is bigger {d} > 15", np.where(abs(diff) > 15), diff.shape)
                        print("\t", tag_mapping[-1], length_ms)
                        print("\t", np.round(a[-5:],0))
                        print("\t", np.round(b[-5:],0))
                        continue
                self.counts.append(length)
                tag_duration = []
                start = 0
                for _, end_ms in tag_mapping:
                    end_time = end_ms / DURATION_SCALER
                    tag_duration.append(end_time - start)
                    start = end_time  # CUMSUM vs DURATION

                pos, bor = pos_prep(torch.FloatTensor(tag_duration[:-1]).to(device))
                position.append(pos)
                border.append(bor)
                weight.append(w)

                out_dur.append(tag_duration)
                inp.append(fbank_feat)
                out_vec.append(tag_vecs)
                out_int.append(tag_ints)
                out_trans.append(transcription)
                out_map.append(tag_mapping)
                self.files.append((label_file, audio_file))
            else:
                print(f"[ERROR] len not match {length} != {len(tag_vecs)} != {len(tag_ints)} \n\t - {label_file}\n\t - {audio_file}")

        self.inp = stack(inp, torch.FloatTensor)
        self.out_vec = stack(out_vec, torch.FloatTensor)
        self.out_int = stack(out_int, torch.LongTensor)
        self.transcription = stack(out_trans, torch.FloatTensor)
        self.out_map = out_map
        self.out_duration = stack(out_dur, torch.FloatTensor)
        self.in_transcription = stack(out_trans, torch.FloatTensor)
        self.key = [uuid.uuid4().urn for i in range(len(inp))]
        self.position = position
        self.border = border
        self.weight = stack(weight, torch.FloatTensor)

        FEATURES = RawField(postprocessing=self.features_batch_process)
        LABEL = RawField(postprocessing=self.features_batch_process)
        TRANSCRIPTION = RawField()
        LABEL_VEC = RawField()
        OUT_MAP = RawField()
        OUT_DUR = RawField(postprocessing=self.features_batch_process)
        IN_TRANS = RawField(postprocessing=self.features_batch_process)
        INDEX = RawField()
        KEY = RawField()
        POSITION = RawField(postprocessing=self.features_batch_process)
        BORDER = RawField(postprocessing=self.features_batch_process)
        WEIGHT = RawField(postprocessing=self.features_batch_process)

        setattr(FEATURES, "is_target", False)
        setattr(LABEL_VEC, "is_target", False)
        setattr(OUT_MAP, "is_target", False)
        setattr(TRANSCRIPTION, "is_target", False)
        setattr(OUT_DUR, "is_target", False)
        setattr(IN_TRANS, "is_target", False)
        setattr(LABEL, "is_target", True)
        setattr(INDEX, "is_target", False)
        setattr(KEY, "is_target", False)
        setattr(POSITION, "is_target", False)
        setattr(BORDER, "is_target", False)
        setattr(WEIGHT, "is_target", False)

        self.fields = {
            "features" : FEATURES,
            "labels" : LABEL,
            "transcription": TRANSCRIPTION,
            "label_vec": LABEL_VEC,
            "out_map": OUT_MAP,
            "out_duration": OUT_DUR,
            "in_transcription": IN_TRANS,
            "index": INDEX,
            "key": KEY,
            "position": POSITION,
            "border": BORDER,
            "weight": WEIGHT,
        }

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx) -> Utterance:
        label_file, audio_file = self.files[idx]
        return Utterance(
            self.inp[idx], self.out_int[idx], self.transcription[idx], self.out_vec[idx],
            self.out_map[idx],
            self.out_duration[idx], self.in_transcription[idx],
            self.position[idx], self.border[idx], self.weight[idx],
            idx, self.key[idx],
            label_file, audio_file,
        )

    @staticmethod
    def features_batch_process(batch) -> UtteranceBatch:
        # this is used when a list of data items is transformed into a batch
        # TODO: could we, should we use pack_padded_sequence
        padded = nn.utils.rnn.pad_sequence(batch, batch_first=True).to(device)
        lens = torch.tensor([len(item) for item in batch]).to(device)
        b, max_len, *f = padded.shape
        return UtteranceBatch(
            padded,
            torch.arange(max_len).expand(len(lens), max_len).to(device) < lens.unsqueeze(1),
            lens
        )

    def batch(self, batch_size=128, sort_key=lambda x: len(x.features), sort=False, shuffle=True, sort_within_batch=True):
        return BucketIterator(self, batch_size=batch_size, sort_key=sort_key, sort=sort, shuffle=shuffle, sort_within_batch=sort_within_batch)


limit = None  # None means unlimited

train_files = DirectMaskDataset.load_csv('train', sa=False)
test_files = DirectMaskDataset.load_csv('test', sa=False)

train_files, train_eval_files = train_test_split(train_files, test_size=500, random_state=42)

train_dataset = DirectMaskDataset(train_files, limit=limit)
train_eval_dataset = DirectMaskDataset(train_eval_files, limit=limit)

test_dataset = DirectMaskDataset(test_files, limit=limit)

toy_dataset = DirectMaskDataset(train_files, limit=1000)


boundary_counts = []
factors = []
# for utterance in chain(train_dataset, train_eval_dataset):
for utterance in test_dataset:
    utterance: Utterance = utterance
    audio_steps, _ = utterance.features.shape
    transcription_steps, _ = utterance.transcription.shape

    factor = audio_steps / transcription_steps
    factors.append(factor)

    boundary_counts.append(len(utterance.out_map))

print(len(boundary_counts), sum(boundary_counts), sum(boundary_counts) / len(boundary_counts))

factors = np.array(factors)
hist, bins = np.histogram(abs(factors))
hist = np.round(hist / len(factors) * 100, 1)

print(sum(factors) / len(factors))
print(1 / (sum(factors) / len(factors)))

POS_TRANSCRIPTION_SCALE = sum(factors) / len(factors)

for prev, tag in sorted(unholy):
    print(prev, '->', tag)

# train_augment_dataset = DirectMaskDataset(train_files, limit=None, augment=True, duplicate=3)
# !7z a TIMIT_CACHE.zip /content/data
# !cp TIMIT_CACHE.zip "/content/drive/My Drive/dataset/TIMIT_CACHE.zip"

# Augmentation

In [ ]:
# !cp "/content/drive/My Drive/dataset/TIMIT_CACHE.zip" TIMIT_CACHE.zip
# !7z x -aos TIMIT_CACHE.zip

In [ ]:
# train_augment_dataset = DirectMaskDataset(train_files, limit=None, augment=True, duplicate=3)

# Utils

In [ ]:
from matplotlib import gridspec
from matplotlib.ticker import FormatStrFormatter

@contextlib.contextmanager
def nullcontext():
    yield None

def load(model, path, ignore: list = None):
    model_dict = model.state_dict()
    pretrained_dict = torch.load(path)
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and (not ignore or not any((i in k) for i in ignore))}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)

def export_model(model, path):
    torch.save(model.state_dict(), path)

# # # # # # # # # # # # # # # # # #

dist = 2
# dist = lambda x, y: norm(x - y, ord=1)
# dist = 'cosine'


def get_aligned_result(result: np.ndarray, labels: np.ndarray) -> np.ndarray:
    step_count, feature_count = result.shape
    labelss = labels
    # result = result + np.random.random_sample(result.shape) / 5
    # labelss = labels + np.random.random_sample(labels.shape) / 5
    # distance, path = __dtw(result, labelss, dist=dist)
    distance, path = slowdtw(result, labelss, dist=dist)
    # warped_result = np.vstack([labels[index,:] for _, index in path])
    # return warped_result, None
    # if len(path) != step_count:
    #     d, cost_matrix, acc_cost_matrix, (result_to_labels, labels_to_result) = adtw(result, labelss, dist=euclidean_norm)
    #     warped_result = np.vstack([labels[index,:] for index in labels_to_result])

    # d, cost_matrix, acc_cost_matrix, (result_to_labels, labels_to_result) = adtw(result, labelss, dist='cosine')
    # warped_result = np.vstack([labels[index,:] for index in labels_to_result])
    # print(len(result_to_labels), len(labels_to_result))
    # path = list(zip(result_to_labels, labels_to_result))

    stack = [None] * step_count
    path_for_id = []
    label_ids = np.argmax(labels, axis=1)
    for index, label_id in path:
        stack[index] = labels[label_id,:]
        path_for_id.append((index, label_ids[label_id]))

    warped_result = np.vstack(stack)
    return warped_result, path_for_id


def generate_duration_transcription(transcriptions: np.ndarray, durations: np.ndarray) -> np.ndarray:
    """Durations be scaled"""
    # durations *= DURATION_SCALER
    # return transcriptions
    ms_per_step = WIN_STEP * 1000
    stack = []
    for feature, duration in zip(transcriptions, durations):
        adj_dur = duration / ms_per_step

        steps = max((adj_dur).round().item(), 1)
        for i in range(int(steps)):
            stack.append(feature)

    transcription_with_duration = np.vstack(stack)
    return transcription_with_duration

def find_borders_pathed(path: list, original_mapping: list):
    # add 0.5 to the border, as it is between the two frames

    borders = []
    last = path[0][1]

    for idx, label_id in path:
        if last != label_id:
            pos = idx - .5
            borders.append(pos * ms_per_step)
        last = label_id
    length = idx + 1

    # group = defaultdict(list)
    # for index, label_id in path:
    #     group[index].append(label_id)

    # length = index + 1
    # for index in range(length):
    #     labels = group[index]
    #     total = len(labels)

    #     for i, label_id in enumerate(labels):
    #         if last != label_id:
    #             pos = index - 0.459 + (.5 / total) * i
    #             borders.append((pos) * ms_per_step)
    #         last = label_id

    # borders.append((length - 0.459) * ms_per_step)  # add last border

    borders_pred = np.array(borders)

    borders_truth = np.array([end for i, (voc, end) in enumerate(original_mapping) if original_mapping[min(i+1, len(original_mapping) - 1)][0] != voc])

    assert (borders_pred.shape == borders_truth.shape) , f"[error] Mapping and Output have same composition {borders_pred.shape} {borders_truth.shape}"

    diff = (borders_truth - borders_pred)
    return None, None, diff


def evaluate_result(model, iterator, lower=True, duration_model=None):
    dtw_errors = []
    detection_errors = []

    print("[standard]" if (duration_model is None) else "[duration model]")
    diff_ranking = []
    diffs = []
    cache_hits = 0
    for batch in iterator:
        features_audio = batch.features.padded
        batch_s, time_s, feat_s = features_audio.shape
        masks_audio = batch.features.masks
        features_transcription = batch.in_transcription.padded
        masks_transcription = batch.in_transcription.masks
        labels = batch.labels.padded

        full_result = model(features_transcription, masks_transcription, features_audio, masks_audio)
        full_result = F.softmax(full_result, dim=2)

        full_result_cls = torch.argmax(full_result, dim=2)
        full_result = full_result.cpu().detach().numpy()

        for i in range(batch_s):
            idx = batch.index[i]
            key = batch.key[i]
            length = batch.labels.lengths[i].cpu().detach().numpy()
            transcription = batch.transcription[i].cpu().detach().numpy()

            result = full_result[i,:length,:]
            result_cls = full_result_cls[i,:length]
            labels_cls = labels[i,:length]

            audio = features_audio[i,:length,:]
            truth = batch.label_vec[i].cpu().detach().numpy()

            if duration_model:
                if key in duration_cache:
                    cache_hits += 1
                    transcription = duration_cache[key]
                else:
                    res_batch = duration_model.forward(batch.transcription[i].unsqueeze(0), None, audio.unsqueeze(0), None)
                    prediction = (res_batch).squeeze(0).detach().cpu().numpy() * DURATION_SCALER
                    transcription = generate_duration_transcription(transcription, prediction)
                    duration_cache[key] = transcription

            # labels is a sequence of vocals in order of their creation with no duration
            warped_result, path = get_aligned_result(result, transcription)
            wmax = np.argmax(warped_result, axis=1)

            error = (labels_cls.cpu().detach().numpy() != wmax[:length]).sum()
            dtw_error = (error / length) * 100

            if dtw_error > 25:
                print(f"danger: dtw_error {dtw_error:.1f}% wrong idx:{idx}")
                print(f"- warped_result: {warped_result.shape}\n- truth:{truth.shape}")
                # f, axarr = plt.subplots(5, figsize=(24, 6))
                # axarr[0].text(5, 5, 'Audio', bbox={'facecolor': 'white', 'pad': 10})
                # axarr[0].imshow(audio.detach().cpu().numpy().T, origin="lower", aspect='auto', cmap=cm.winter)
                # axarr[1].imshow(truth.T, origin="lower", aspect='auto', cmap=cm.winter)
                # axarr[1].text(5, 5, 'truth', bbox={'facecolor': 'white', 'pad': 10})
                # axarr[2].imshow(transcription.T, origin="lower", aspect='auto', cmap=cm.winter)
                # axarr[2].text(5, 5, 'transcription', bbox={'facecolor': 'white', 'pad': 10})
                # axarr[3].imshow(warped_result.T, origin="lower", aspect='auto', cmap=cm.winter)
                # axarr[3].text(5, 5, 'warped_result', bbox={'facecolor': 'white', 'pad': 10})
                # axarr[4].imshow(result.T, origin="lower", aspect='auto', cmap=cm.winter)
                # axarr[4].text(5, 5, 'result - probs', bbox={'facecolor': 'white', 'pad': 10})


            if dtw_error < 1 and lower:
                print(f"winner: dtw_error {dtw_error:.1f}% good")
            dtw_errors.append(dtw_error)

            error = (labels_cls != result_cls).sum().cpu().detach().numpy()
            detection_errors.append((error / length) * 100)

            try:
                if path:
                    a, b, d = find_borders_pathed(path, batch.out_map[i])
                else:
                    a, b, d = find_borders(wmax, batch.out_map[i])
            except Exception as e:
                print("[Exception]")
                # print(np.argmax(transcription, axis=1))
                print(labels_cls.shape, wmax.shape, transcription.shape, len(batch.out_map[i]))
                raise e
            diff_ranking.append((abs(d.max()), idx))
            diffs.append(d)

    diff_ranking = sorted(diff_ranking, key=lambda x: x[0])
    print(diff_ranking[-5:])
    print(f"cache_hits: {cache_hits}")
    diff = np.concatenate(diffs)
    return dtw_errors, detection_errors, diff

def display_error(errors, name=""):
    print(f"[{name}]AVERAGE ERROR: {sum(errors) / len(errors):.2f}% COUNT:{len(errors)}")

def display_diff(errors, name="", unit='ms', plotting=False):
    errors = errors.copy()
    hist, bins = np.histogram(abs(errors), bins=[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60,65,70,75,80,85,90, 95, 100, 105, 9999])
    hist = np.round(hist / len(errors) * 100, 2)
    hist = np.cumsum(hist)

    print(f"[{name}] DIFF abs mean: {abs(errors).mean():.2f}{unit} ({errors.mean():.2f}) min:{abs(errors).min():.2f}{unit} max:{abs(errors).max():.2f}{unit}")
    rows = list(zip(hist, bins, bins[1:]))
    for R  in zip(rows[::2], rows[1::2]):
        s = ""
        for h, b, e in R:
            s += f"\t{h:.2f}%\t < {e:.0f}{unit}\t"
        print(s)

    print(*[f'{h:2.2f}' for h, b, e in rows][:-2], "", sep="% ")
    # print([e for h, b, e in rows])

    if plotting:
        f, axarr = plt.subplots(1, 2, figsize=(10, 3))
        axarr[0].bar(range(len(bins)-1), hist, )
        axarr[0].set_xticklabels(bins, fontdict=None, minor=False)
        axarr[1].hist(np.clip(errors, -70, 70), bins=5)

def draw_duration(model, dataset, index):
    model.eval()
    inp = dataset[index]
    prediction = model.forward(inp.in_transcription.unsqueeze(0), None, inp.features.unsqueeze(0), None).squeeze(0).detach().cpu().numpy() * DURATION_SCALER

    inputs = inp.features.detach().cpu().numpy()
    transcription_truth = dataset.out_vec[index].detach().cpu().numpy()
    truth = inp.out_duration.detach().cpu().numpy() * DURATION_SCALER

    total_duration = inputs.shape[0] * ms_per_step
    print(total_duration, sum(prediction))
    prediction = prediction / sum(prediction) * total_duration
    transcription_with_duration = generate_duration_transcription(inp.in_transcription.detach().cpu().numpy(), prediction)

    f, axarr = plt.subplots(4, figsize=(8, 8))
    axarr[0].title.set_text('1. Audio input')
    axarr[1].title.set_text('2. Duration predictions')
    axarr[2].title.set_text('3. Phoneme occurrence predicted by scaling transcription with durations')
    axarr[3].title.set_text('4. Phoneme occurrence ground truth')

    axarr[0].imshow(inputs.T, origin="lower", aspect='auto', cmap=cm.winter)
    axarr[1].plot(prediction, 'r', label="Predicted durations")
    axarr[1].plot(truth, 'g', label="Actual durations")
    axarr[1].legend(loc='upper left')

    axarr[2].imshow(transcription_with_duration.T, origin="lower", aspect='auto')
    axarr[3].imshow(transcription_truth.T, origin="lower", aspect='auto')

    axarr[0].set_ylabel('Audio features')
    axarr[0].set_xlabel('Audio frame index')
    axarr[1].set_ylabel('Duration in ms')
    axarr[1].set_xlabel('Phoneme index')
    axarr[2].set_xlabel('Audio frame index')
    axarr[2].set_ylabel('Phoneme one-hot encoding')
    axarr[3].set_xlabel('Audio frame index')
    axarr[3].set_ylabel('Phoneme one-hot encoding')

    f.tight_layout()

def draw_audio(model, dataset, index):
    model.eval()
    ds = dataset[index]
    features_audio = ds.features
    transcription_truth = dataset.out_vec[index].detach().cpu().numpy()

    trans = ds.in_transcription

    res_batch = model.forward(trans.unsqueeze(0), None, features_audio.unsqueeze(0), None)

    inputs = ds.features.detach().cpu().numpy()

    prediction = (res_batch).squeeze(0).detach().cpu().numpy()
    truth = (ds.out_duration).detach().cpu().numpy()  * DURATION_SCALER
    trans = trans.detach().cpu().numpy()

    f, axarr = plt.subplots(4, figsize=(10, 4))
    axarr[0].imshow(inputs.T, origin="lower", aspect='auto', cmap=cm.winter)
    axarr[1].plot(truth, 'g', label="Truth")
    axarr[1].plot(prediction, 'r', label="Prediction")
    axarr[1].legend(loc="upper right")
    axarr[2].imshow(transcription_with_duration.T, origin="lower", aspect='auto', cmap=cm.winter)
    axarr[3].imshow(transcription_truth.T, origin="lower", aspect='auto', cmap=cm.winter)

def show_audio(model, dataset, name, plot_only = False, duration_model=None):
    print(f"\n[{name}]")
    model.eval()
    ### show one
    i = 36
    inp = test_dataset[i].features
    out_vec = test_dataset.out_vec[i]
    transcription = test_dataset.in_transcription[i]
    _res = model.forward(transcription.unsqueeze(0), None, inp.unsqueeze(0), None)
    _res = F.softmax(_res, dim=2)
    result = _res[0,:,:]
    result_maximized = result.clone().detach().cpu().numpy()
    ids = np.argmax(result_maximized, axis=1)
    result_maximized = result_maximized * 0
    for t, i in enumerate(ids):
        result_maximized[t, i] = 1
    warped_result, path = get_aligned_result(result.detach().cpu().numpy(), transcription.detach().cpu().numpy())

    f, axarr = plt.subplots(4, figsize=(12, 12), sharex=True)
    axarr[0].imshow(inp.cpu().numpy().T, origin="lower", aspect='auto')#, cmap=cm.winter)
    axarr[1].imshow(result.detach().cpu().numpy().T, origin="lower", aspect='auto')#, cmap=cm.winter)
    axarr[2].imshow(result_maximized.T, origin="lower", aspect='auto')#, cmap=cm.winter)
    axarr[3].imshow(out_vec.cpu().numpy().T, origin="lower", aspect='auto')#, cmap=cm.winter)
    axarr[0].title.set_text('Audio input')
    axarr[1].title.set_text('CTC phoneme probabilities')
    axarr[2].title.set_text('CTC most probable phoneme')
    axarr[3].title.set_text('Phoneme ground truth')

    f, axarr = plt.subplots(3, figsize=(12, 8), sharex=True)
    axarr[0].imshow(result.detach().cpu().numpy().T, origin="lower", aspect='auto')#, cmap=cm.winter)
    axarr[1].imshow(warped_result.T, origin="lower", aspect='auto')#, cmap=cm.winter)
    axarr[2].imshow(out_vec.cpu().numpy().T, origin="lower", aspect='auto')#, cmap=cm.winter)
    axarr[0].title.set_text('CTC phoneme probabilities')
    axarr[1].title.set_text('Transcription phoneme sequence alligned over CTC result with DTW')
    axarr[2].title.set_text('Phoneme ground truth')

    if plot_only: return

    # # # # # #
    # difference percentages
    dataset_iter = BucketIterator(dataset, batch_size=64, sort_key=lambda x: len(x.features), sort=False, shuffle=True, sort_within_batch=True)
    dtw_errors, detection_errors, diff = evaluate_result(model, dataset_iter, lower=False, duration_model=duration_model)
    display_error(dtw_errors, "DETECTION+DTW")
    display_error(detection_errors, "DETECTION")
    display_diff(diff, name, plotting=True)

def show_duration(model, dataset, sample_size=2000):
    model.eval()
    print("dataset len", len(dataset))

    # draw_duration(model, dataset, 0)
    # # # # # #
    # difference percentages
    diffs = []
    sums = []
    for i in sample(range(len(dataset)), min(len(dataset), sample_size)):
        inp = dataset[i].in_transcription
        inp_audio = dataset[i].features

        out = dataset[i].out_duration * DURATION_SCALER  # durations have been scaled with DURATION_SCALER
        res = model.forward(inp.view(1, *inp.shape), None, inp_audio.unsqueeze(0), None) * DURATION_SCALER

        resc = torch.cumsum(res, dim=1).view(-1)
        res = res.detach().view(-1).cpu().numpy()

        outc = torch.cumsum(out, dim=0)
        out = out.detach().cpu().numpy()

        diff = out-res
        if diff.max() > 1000:
            print(i, diff.max())
            draw_duration(model, dataset, i)
            # continue
        diffs.append(diff)
        sums.append((outc-resc).detach().cpu().numpy())

    diff = np.concatenate(diffs)
    sums = np.concatenate(sums)
    display_diff(diff, "duration", plotting=True)
    display_diff(sums, "position")

show_duration_og = show_duration

def show_position(model, dataset, duration_combined_model=None, sample_size=2000, report_error=750, skip=False):
    model.eval()
    if duration_combined_model is not None:
        duration_combined_model.eval()

    pos_map = pos_prep.pe
    c = torch.cumsum(torch.ones(2 ** 14), 0).unsqueeze(1) - 1
    print("dataset len", len(dataset))

    diffs = []
    att = Attention(POS_DIM)

    for idx in sample(range(len(dataset)), min(len(dataset), sample_size)):
        inp: Utterance = dataset[idx]

        audio = inp.features
        transcription = inp.in_transcription
        border = inp.border

        length = audio.shape[0] + 0
        pos_feature = pos_map[:length,:]

        borders_predicted = model(transcription.unsqueeze(0), None, audio.unsqueeze(0), None)[0]

        # b = borders_predicted.detach().cpu().numpy() * ms_per_step
        # prev = 0
        # switched = False
        # for i, v in enumerate(b):
        #     if v < prev:
        #         switched = True
        #         after = b[i+1] if i + 1 < len(b) else prev + 10
        #         v = (prev + after) / 2
        #     b[i] = v
        #     prev = v

        if duration_combined_model is None:
            new = borders_predicted.detach().cpu().numpy() * ms_per_step
            switched = False
            prev = 0
            for i, v in enumerate(new):
                if abs(v - prev) > 500 or  v < prev:
                    after = new[i+1] if i + 1 < len(new) else prev
                    v = (prev + after) / 2
                    switched = True
                new[i] = v
                prev = v
            b = new
        else:
            duration = None
            prediction_position = borders_predicted.detach().cpu().numpy() * ms_per_step
            new = prediction_position.copy()
            switched = False
            prev = 0
            for i, v in enumerate(new):
                after = new[i+1] if i + 1 < len(new) else v
                if v < prev or v > after:
                    if duration is None:
                        duration = (duration_combined_model(transcription.unsqueeze(0), None, audio.unsqueeze(0), None) * DURATION_SCALER).view(-1).detach().cpu().numpy()[:-1]
                    v =  prev + duration[i - 1]
                    switched = True
                new[i] = v
                prev = v
            b = new

        diff = (border.detach().cpu().numpy() * ms_per_step - b)
        if np.abs(diff).max() > report_error:
            print(f"[id:{idx:3d}]  [{diff.min():5.0f} {diff.max():5.0f}]  {length:4d} {switched}")
            if skip:
                continue
        if switched and skip:
            continue

        diffs.append(diff)

    diff = np.concatenate(diffs)
    display_diff(diff, "position", plotting=True)


def location_fix(positions, truth, durations, end_of_audio):
    switched = False

    prev = 0
    difos = []
    visited = []
    for _ in range(10):
        # worst_diff, worst_index = max([abs(v - Y), i] for i, (v, Y) in enumerate(zip(positions, truth) )if i not in visited)
        worst_diff, worst_index = 0, 0
        for i, v in enumerate(positions):
            if i in visited: continue
            prev = positions[i-1] if i - 1 >= 0 else 0
            after = positions[i+1] if i + 1 < len(positions) else end_of_audio
            Y = (prev + after - 0.0001) / 2
            Y = prev + durations[i]
            diff = abs(v - Y)
            if diff > worst_diff:
                worst_diff, worst_index = diff, i

        if worst_diff < 300:
            continue
        visited.append(worst_index)
        difos.append([worst_diff, [positions[worst_index], truth[worst_index]], worst_index])

        i = worst_index
        v = positions[i]
        prev = positions[i-1] if i - 1 >= 0 else 0
        after = positions[i+1] if i + 1 < len(positions) else end_of_audio

        if prev < after:
            v = (prev + after - 0.0001) / 2
        else:
            v = prev + 0.001

        positions[worst_index] = v

    prev = 0
    for i, _ in enumerate(positions):
        v = positions[i]

        if v < prev:
            switched = True
            after = positions[i+1] if i + 1 < len(positions) else prev + 0.01

            if v < prev:
                if prev < after:
                    v = (prev + after - 0.0001) / 2
                else:
                    v = prev + 0.001

        positions[i] = v
        prev = v

    prev = 0
    for i, v in enumerate(positions):
        assert v >= prev, f"This should never happen! {i}"
        prev = v
    return positions, difos

def show_position_batched(model, dataset, duration_combined_model=None, report_error=750, skip=False, plotting=False, dumb_limit=500):
    model.eval()
    if duration_combined_model is not None:
        duration_combined_model.eval()

    pos_map = pos_prep.pe
    c = torch.cumsum(torch.ones(2 ** 14), 0).unsqueeze(1) - 1
    print("dataset len", len(dataset))

    diffs = []
    label_ids = []
    att = Attention(POS_DIM)

    for batch in dataset.batch(32):
        features_audio = batch.features.padded
        masks_audio = batch.features.masks

        features_transcription = batch.in_transcription.padded
        masks_transcription = batch.in_transcription.masks

        borders = batch.border.padded.cpu().detach().numpy()
        border_lengths = batch.border.lengths.cpu().detach().numpy()

        batch_s, time_s, feat_s = features_audio.shape

        borders_predicted = model(features_transcription, masks_transcription, features_audio, masks_audio).cpu().detach().numpy()

        if duration_combined_model is not None:
            duration_batch = (duration_combined_model(features_transcription, masks_transcription, features_audio, masks_audio) * DURATION_SCALER).detach().cpu().numpy()

        for i in range(batch_s):
            label_id = [l_id for l_id, ms in batch.out_map[i]]
            idx = batch.index[i]
            key = batch.key[i]

            length = border_lengths[i]

            predicted_border = borders_predicted[i,:length]
            truth_border = borders[i,:length]
            end_of_audio = length * ms_per_step

            b = predicted_border * ms_per_step

            # b, difos = location_fix(b.copy(), truth_border * ms_per_step, duration_batch[i].reshape(-1), end_of_audio)
            switched = False
            prev = 0
            if duration_combined_model is not None:
                duration = duration_batch[i].reshape(-1)

            for i, v in enumerate(b):
                v = b[i]
                if v < prev:
                    switched = True
                    after = b[i+1] if i + 1 < len(b) else end_of_audio # prev + 0.01 # end of file? end_of_audio
                    if duration_combined_model is not None:
                        v = after - duration[i - 1]

                    if v < prev:
                        if prev < after:
                            v = (prev + after - 0.0001) / 2
                        else:
                            v = prev + 0.001
                b[i] = v
                prev = v

            prev = 0
            for i, v in enumerate(b):
                assert v >= prev, f"This should never happen! {i}"
                prev = v

            diff = (truth_border * ms_per_step - b)
            if np.abs(diff).max() > report_error:
                print(f"[id:{idx:3d}]  [{diff.min():5.0f} {diff.max():5.0f}]  {length:4d} {switched}")
                # print(*difos, sep="\n")
            #     if skip:
            #         continue
            # if switched and skip:
            #     continue

            diffs.append(diff)
            label_ids.append(label_id)

    diff = np.concatenate(diffs)
    label_ids = np.concatenate(label_ids)

    phoneme_map = defaultdict(list)
    for pdur, pid in zip(diff, label_ids):
        phoneme_map[pid].append(abs(pdur))

    for func in [np.max, np.mean]:
        print(func)
        mean_phoneme_dur = sorted([[f"{KNOWN_LABELS[pid].ljust(4)}", func(val) ] for pid, val in phoneme_map.items()], key=lambda x: x[1])
        print(len(mean_phoneme_dur))
        for row in zip(mean_phoneme_dur[::5], mean_phoneme_dur[1::5], mean_phoneme_dur[2::5], mean_phoneme_dur[3::5], mean_phoneme_dur[4::5]):
            for i, (p, c) in enumerate(row):
                print(f"{p} {c:5.2f}ms".ljust(20), end="", sep="")
            print()
        for i, (p, c) in enumerate(mean_phoneme_dur[-4:]):
            print(f"{p} {c:5.2f}ms".ljust(20), end="", sep="")
        print("\n.")
        figure()
        plt.plot(*zip(*[[len(val), func(val)] for pid, val in phoneme_map.items()]), 'wo')
        for pid, val in phoneme_map.items():
            plt.annotate(KNOWN_LABELS[pid], xy=(len(val), func(val)))
        plt.xlabel('Occurence count', fontsize=13)
        plt.ylabel(f'Mean error' if func is np.mean else "Max error", fontsize=13)
        plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%dms'))
        plt.show()

    print("TOTAL", np.abs(diff).sum())
    display_diff(diff, "position", plotting=plotting)

# + + + + ++ + + + + + + + + ++ + + + + + + + + + + + + + + + ++  +

def explore_inherit_border_error(dataset):
    print("""[PLOT] How much do the borders of processed audio transcription differ from the original timesteps?""")
    time_iter = BucketIterator(dataset, batch_size=64, sort_key=lambda x: len(x.features), sort=False, shuffle=True, sort_within_batch=True)

    diffs = []
    for batch in time_iter:
        batch_s = len(batch.label_vec)
        for i in range(batch_s):
            original = batch.label_vec[i].cpu().numpy()
            trans = batch.transcription[i].cpu().numpy()
            original_ids = np.argmax(original, axis=1)
            mapping = batch.out_map[i]

            try:
                a, b, d = find_borders(original_ids, mapping)
                diffs.append(d)
            except Exception as e:
                print(e)
                arr = (np.where(original_ids[:-1] != original_ids[1:])[0] + 0.5) * WIN_STEP * 1000
                last = original_ids.shape[0] * WIN_STEP * 1000
                arr = np.append(arr, last)
                print(len(mapping), len(dedupe(original_ids)), arr.shape, trans.shape, original_ids[:10], original_ids[-10:])
                prev = 0
                for (voc, t), oid in zip(mapping, dedupe(original_ids)+[99, 99, 99], ):
                    print(f"{int(voc)}-{int(oid)} \t{t:.0f}\t{t - prev:.0f}")
                    prev = t

    diff = np.concatenate(diffs)
    display_diff(diff, "trans. vs. timesteps", plotting=True)

def draw_counts(counts, name):
    print(f'[dataset rows]{name}: {len(counts)}')
    f, axarr = plt.subplots(1, 2, figsize=(10, 3))
    axarr[0].hist(counts, bins=25)
    axarr[1].hist(counts, bins=10);


# explore_inherit_border_error(train_dataset)

# draw_counts(train_dataset.counts, 'train')  # 4573
# draw_counts(test_dataset.counts, 'test')  # 1656
# [dataset rows]train: 4606 sa
# [dataset rows]train: 3684 n sa

# no changes
# 56.32% 82.07% 90.99% 94.64% 96.63% 97.78% 98.46% 98.88% 99.13% 99.30% 99.44% 99.51% 99.58% 99.62% 99.65% 99.67% 99.69% 99.71% 99.72% 99.74%
# special
# 56.14% 81.81% 90.71% 94.35% 96.32% 97.46% 98.13% 98.55% 98.80% 98.97% 99.11% 99.18% 99.25% 99.29% 99.32% 99.35% 99.37% 99.39% 99.40% 99.42%
# vanilla
# 56.32% 82.07% 90.99% 94.63% 96.61% 97.76% 98.43% 98.85% 99.11% 99.28% 99.42% 99.49% 99.56% 99.60% 99.63% 99.65% 99.67% 99.69% 99.70% 99.72%
# activation
# 56.31% 82.06% 90.97% 94.61% 96.59% 97.74% 98.41% 98.83% 99.08% 99.25% 99.39% 99.46% 99.53% 99.57% 99.61% 99.63% 99.65% 99.67% 99.68% 99.70%
# duration
# 56.14% 81.81% 90.71% 94.35% 96.32% 97.46% 98.13% 98.55% 98.80% 98.97% 99.11% 99.18% 99.25% 99.29% 99.32% 99.35% 99.37% 99.39% 99.40% 99.42%
# combined
# 56.31% 82.06% 90.98% 94.62% 96.60% 97.75% 98.43% 98.85% 99.11% 99.28% 99.42% 99.49% 99.57% 99.61% 99.64% 99.66% 99.68% 99.70% 99.71% 99.73%
# final
# 56.31% 82.06% 90.98% 94.62% 96.60% 97.75% 98.43% 98.85% 99.11% 99.28% 99.42% 99.49% 99.57% 99.61% 99.64% 99.66% 99.68% 99.70% 99.71% 99.73%
# for dumb in range(50, 1000, 50):
# show_position_batched(position_model.with_gradient, test_dataset, duration_combined_model=duration_combined_model, dumb_limit=250)
# show_position_batched(position_model.with_gradient, test_dataset, duration_combined_model=None, dumb_limit=250)
# show_position_batched(position_model.with_gradient, test_dataset, duration_combined_model=None, skip=True)

# show_position_batched(position_model.with_gradient, train_dataset, duration_combined_model=duration_combined_model, dumb_limit=500)

# 51.07% 78.61% 89.44% 93.93% 96.40% 97.71% 98.44% 98.90% 99.17% 99.38% 99.49% 99.57% 99.64% 99.69% 99.72% 99.75% 99.77% 99.78% 99.79% 99.80%
show_position_batched(position_model.with_gradient, test_dataset)

# Train

In [ ]:
class MaskedLoss(nn.Module):
    mse = nn.MSELoss()
    def forward(self, pred, target, mask):
        # "flatten" all logits and targets by putting all subsequences together
        # pred = torch.log1p(pred).contiguous().view(-1)
        # target = torch.log1p(target).contiguous().view(-1)
        pred = torch.log1p(pred).contiguous().view(-1)
        target = torch.log1p(target).contiguous().view(-1)
        # pred = torch.log10(1 + pred).contiguous().view(-1)
        # target = torch.log10(1 + target).contiguous().view(-1)
        mask = mask.view(-1)
        pred = (mask * pred.T).T
        return self.mse(pred, target)

class LabelSmoothingLossAudio(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super().__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target, mask):
        # print(pred.shape, target.shape, mask.shape)
        pred = pred.log_softmax(dim=self.dim)
        # pred: torch.Size([32, 512, 61]) target: torch.Size([32, 512]) Mask:torch.Size([32, 512])
        pred = (mask * pred.T).T
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

class PositionShuffleLoss(nn.Module):
    mse = nn.MSELoss()
    cos = nn.CosineSimilarity(dim=2, eps=1e-6)
    w = torch.FloatTensor([min(1.05 ** i, 10) for i in range(POS_DIM)]).to(device)

    def forward(self, pred, target, mask):
        pred = torch.mul(pred, mask.unsqueeze(2))
        idx = random.randint(0, 5)
        # idx = 8
        if idx == 0:
            return self.mse(pred, target)
        # elif idx == 1:
            # return self.mse(pred * self.w, target * self.w)
        elif idx == 2:
            return self.mse(pred, target) * (2. - self.cos(pred, target)).mean()
        else:
            return (1. - self.cos(pred, target)).mean()

class PositionMSELoss(nn.Module):
    mse = nn.MSELoss()
    w = torch.FloatTensor([min(1.05 ** i, 10) for i in range(POS_DIM)]).to(device)

    def forward(self, pred, target, mask):
        pred = torch.mul(pred, mask.unsqueeze(2))
        idx = random.randint(0, 1)
        if idx == 0:
            return self.mse(pred, target)
        elif idx == 1:
            return self.mse(pred * self.w, target * self.w)

class CosineLoss(nn.Module):
    cos = nn.CosineSimilarity(dim=2, eps=1e-6)

    def forward(self, pred, target, mask):
        pred = torch.mul(pred, mask.unsqueeze(2))
        return (1. - self.cos(pred, target)).mean()

class MaskedMSE(nn.Module):
    mse = nn.MSELoss()
    def forward(self, pred, target, mask):
        pred = torch.mul(pred, mask)
        return self.mse(pred, target)

class MaskedSoftL1(nn.Module):
    loss = nn.SmoothL1Loss()

    def __init__(self, factor=5):
        super().__init__()
        self.factor = factor

    def forward(self, pred, target, mask):
        pred = torch.mul(pred, mask)
        return self.loss(pred / self.factor, target / self.factor)

class LabelSmoothingLossAudio(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super().__init__()
        assert (smoothing >= 0.0 and smoothing <= 1.0)

        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target, mask):
        # print(pred.shape, target.shape, mask.shape)
        pred = pred.log_softmax(dim=self.dim)
        # pred: torch.Size([32, 512, 61]) target: torch.Size([32, 512]) Mask:torch.Size([32, 512])
        pred = (mask * pred.T).T
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


def position_encode_trainer(batch: UtteranceBatch, model: nn.Module, loss_function: nn.Module):
    features_audio = batch.features.padded
    masks_audio = batch.features.masks
    features = batch.in_transcription.padded
    masks = batch.in_transcription.masks
    target = batch.position.padded
    target_mask = batch.position.masks

    result = model(features, masks, features_audio, masks_audio)
    return loss_function(result, target, target_mask)


def position_gradient_trainer(batch: UtteranceBatch, model: nn.Module, loss_function: nn.Module):
    features_audio = batch.features.padded
    masks_audio = batch.features.masks
    features = batch.in_transcription.padded
    masks = batch.in_transcription.masks

    result = model(features, masks, features_audio, masks_audio)

    target = batch.border.padded
    target_mask = batch.border.masks
    return loss_function(result, target, target_mask, batch.weight.padded)

def audio_detection_trainer(batch: UtteranceBatch, model: nn.Module, loss_function: nn.Module):
    features_audio = batch.features.padded
    masks_audio = batch.features.masks
    features_transcription = batch.in_transcription.padded
    masks_transcription = batch.in_transcription.masks
    target = batch.labels.padded

    result = model(features_transcription, masks_transcription, features_audio, masks_audio)

    batch_s, time_s, feat_s = result.shape
    # "flatten" all logits and targets by putting all subsequences together
    flattened_result = result.contiguous().view(batch_s * time_s, -1)
    flattened_targets = target.contiguous().view(-1)
    flattened_masks = masks_audio.view(-1)

    return loss_function(flattened_result, flattened_targets, flattened_masks)


def duration_trainer(batch: UtteranceBatch, model: nn.Module, loss_function: nn.Module):
    features_audio = batch.features.padded
    masks_audio = batch.features.masks
    features = batch.in_transcription.padded
    masks = batch.in_transcription.masks

    result = model(features, masks, features_audio, masks_audio)

    target = batch.out_duration.padded
    target_mask = batch.out_duration.masks
    return loss_function(result.squeeze(2), target, target_mask)


def train(model, num_epochs, data_iter, eval_iter=None, loss_function=CosineLoss(), train_function=position_encode_trainer, lr_decay=0.9, lr=0.001, weight_decay=1e-5, repreat=0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    # optimizer = torch.optim.ASGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    print(optimizer)

    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=lr_decay)
    eval_iter = eval_iter or data_iter
    eval_iter = [eval_iter] if type(eval_iter) is not list else eval_iter

    for epoch in range(1, num_epochs+1):
        for e_iter in eval_iter:
            train_acc = evaluate(model, e_iter, train_function, loss_function)

        print("Starting epoch %d, learning rate is %f" % (epoch, lr_scheduler.get_lr()[0]))
        errors = []
        for batch in data_iter:
            model.zero_grad()
            model.train()
            optimizer.zero_grad()
            loss = train_function(batch, model, loss_function)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
            optimizer.step()
            errors.append( (loss.clone().detach().cpu().numpy(), batch) )

        if repreat:
            errors = list(sorted(errors, key=lambda x: x[0])[-3:])
            print("  ", *[round(s.item(), 3) for s,_ in errors])
            for i in range(repeat):
                for s, batch in errors:
                    model.zero_grad()
                    model.train()
                    optimizer.zero_grad()
                    loss = train_function(batch, model, loss_function)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
                    optimizer.step()

        lr_scheduler.step()

    for e_iter in eval_iter:
        train_acc = evaluate(model, e_iter, train_function, loss_function)

def evaluate(model, data_iter, train_function=position_encode_trainer, loss_function=CosineLoss()):
    model.eval()
    total_loss = 0
    size = 0
    with torch.no_grad():
        for batch in data_iter:
            total_loss += abs(train_function(batch, model, loss_function).item())
            size += 1
    print(f"  Evaluation[{getattr(data_iter, 'prefix', '')}] - avg_loss: {total_loss/size:.7f} count:{size} Total loss:{total_loss:.7f}")


# - - - - - - - - - - - - - -
# - - - - - - - - - - - - - -
# - - - - - - - - - - - - - -



# Attention

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, scale=1, max_len=2048):
        super(PositionalEncoding, self).__init__()
        self.scale = scale
        self.dropout = nn.Dropout(p=dropout)
        if not self.scale:
            return
        max_len = int(max_len * scale)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        m = nn.Upsample(scale_factor=(1. / scale, 1), mode='bilinear', align_corners=True)
        shape = pe.shape
        pe = pe.view(1, 1, *shape)
        pe = m(pe).view(-1, d_model)

        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        # print("Dropout", self.dropout)

    def forward(self, x):
        if not self.scale:
            return x
        x = x.transpose(0, 1)
        x = x + self.pe[:x.size(0), :]
        x = x.transpose(0, 1)
        return self.dropout(x)

class Attention(nn.Module):
    r"""
    https://github.com/IBM/pytorch-seq2seq/blob/master/seq2seq/models/attention.py
    Applies an attention mechanism on the output features from the decoder.
    .. math::
            \begin{array}{ll}
            x = context*output \\
            attn = exp(x_i) / sum_j exp(x_j) \\
            output = \tanh(w * (attn * context) + b * output)
            \end{array}
    Args:
        dim(int): The number of expected features in the output
    Inputs: output, context
        - **output** (batch, output_len, dimensions): tensor containing the output features from the decoder.
        - **context** (batch, input_len, dimensions): tensor containing features of the encoded input sequence.
    Outputs: output, attn
        - **output** (batch, output_len, dimensions): tensor containing the attended output features from the decoder.
        - **attn** (batch, output_len, input_len): tensor containing attention weights.
    Attributes:
        linear_out (torch.nn.Linear): applies a linear transformation to the incoming data: :math:`y = Ax + b`.
        mask (torch.Tensor, optional): applies a :math:`-inf` to the indices specified in the `Tensor`.
    Examples::
         >>> attention = seq2seq.models.Attention(256)
         >>> context = Variable(torch.randn(5, 3, 256))
         >>> output = Variable(torch.randn(5, 5, 256))
         >>> output, attn = attention(output, context)
    """
    def __init__(self, dim):
        super(Attention, self).__init__()
        self.dim = dim
        if self.dim:
            self.linear_out = nn.Linear(dim * 2, dim)

    def forward(self, output, context, mask = None):
        # https://arxiv.org/abs/1706.03762
        # context & mask is what we attend to
        batch_size, hidden_size, input_size = output.size(0), output.size(2), context.size(1)
        # (batch, out_len, dim) * (batch, in_len, dim) -> (batch, out_len, in_len)
        # matrix by matrix product https://pytorch.org/docs/stable/torch.html#torch.bmm
        attn = torch.bmm(output, context.transpose(1, 2))
        # TODO: scale step missing?

        if mask is not None:
            attn.data.masked_fill_(~mask.unsqueeze(1), -float('inf'))

        attn = F.softmax(attn.view(-1, input_size), dim=1).view(batch_size, -1, input_size)
        if not self.dim:
            return attn

        mix = torch.bmm(attn, context)  # (batch, out_len, in_len) * (batch, in_len, dim) -> (batch, out_len, dim)
        combined = torch.cat((mix, output), dim=2)  # concat -> (batch, out_len, 2*dim)

        # output -> (batch, out_len, dim)
        output = torch.tanh(self.linear_out(combined.view(-1, 2 * hidden_size))).view(batch_size, -1, hidden_size)
        return output, attn

class Attention(nn.Module):
    r"""
    https://github.com/IBM/pytorch-seq2seq/blob/master/seq2seq/models/attention.py
    Applies an attention mechanism on the output features from the decoder.
    .. math::
            \begin{array}{ll}
            x = context*output \\
            attn = exp(x_i) / sum_j exp(x_j) \\
            output = \tanh(w * (attn * context) + b * output)
            \end{array}
    Args:
        dim(int): The number of expected features in the output
    Inputs: output, context
        - **output** (batch, output_len, dimensions): tensor containing the output features from the decoder.
        - **context** (batch, input_len, dimensions): tensor containing features of the encoded input sequence.
    Outputs: output, attn
        - **output** (batch, output_len, dimensions): tensor containing the attended output features from the decoder.
        - **attn** (batch, output_len, input_len): tensor containing attention weights.
    Attributes:
        linear_out (torch.nn.Linear): applies a linear transformation to the incoming data: :math:`y = Ax + b`.
        mask (torch.Tensor, optional): applies a :math:`-inf` to the indices specified in the `Tensor`.
    Examples::
         >>> attention = seq2seq.models.Attention(256)
         >>> context = Variable(torch.randn(5, 3, 256))
         >>> output = Variable(torch.randn(5, 5, 256))
         >>> output, attn = attention(output, context)
    """
    def __init__(self, dim):
        super(Attention, self).__init__()
        self.dim = dim
        if self.dim:
            self.linear_out = nn.Linear(dim * 2, dim)

    def forward(self, output, mask_output, context, mask_context):
        # https://arxiv.org/abs/1706.03762
        # context & mask is what we attend to
        batch_size, hidden_size, input_size = output.size(0), output.size(2), context.size(1)
        # (batch, out_len, dim) * (batch, in_len, dim) -> (batch, out_len, in_len)
        # matrix by matrix product https://pytorch.org/docs/stable/torch.html#torch.bmm
        attn = torch.bmm(output, context.transpose(1, 2))
        # TODO: scale step missing?

        if mask_context is not None:
            if mask_output is not None:
                attn = attn.transpose(1, 2)
                attn.data.masked_fill_(~mask_output.unsqueeze(1), 0)
                attn = attn.transpose(1, 2)

            attn.data.masked_fill_(~mask_context.unsqueeze(1), -float('inf'))

        attn = F.softmax(attn.view(-1, input_size), dim=1).view(batch_size, -1, input_size)
        if not self.dim:
            return attn

        mix = torch.bmm(attn, context)  # (batch, out_len, in_len) * (batch, in_len, dim) -> (batch, out_len, dim)
        combined = torch.cat((mix, output), dim=2)  # concat -> (batch, out_len, 2*dim)

        # output -> (batch, out_len, dim)
        output = torch.tanh(self.linear_out(combined.view(-1, 2 * hidden_size))).view(batch_size, -1, hidden_size)
        return output, attn

class Decoder(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, num_layers=2, dropout=0.1, time_scale=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.attn = Attention(hidden_size)

        self.pos_encode = PositionalEncoding(hidden_size, dropout, scale=time_scale)

    def forward(self, previous, mask_trans, hidden_state, encoder_outputs, mask_audio):
        rnn_output, hidden_state = self.gru(previous, hidden_state)

        rnn_output = self.pos_encode(rnn_output)

        # print(rnn_output.shape if rnn_output is not None else rnn_output)
        # print(mask_trans.shape if mask_trans is not None else mask_trans)
        # print(encoder_outputs.shape if encoder_outputs is not None else encoder_outputs)
        # print(mask_audio.shape if mask_audio is not None else mask_audio)
        output, attn = self.attn(rnn_output, mask_trans, encoder_outputs, mask_audio)
        output = self.out(output)
        return output, hidden_state

class Encoder(nn.Module):
    def __init__(self, hidden_size, embedding_size, out_dim=None, num_layers=2, dropout=0.1, time_scale=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.batchnorm = nn.BatchNorm1d(embedding_size)
        # Embedding layer that will be shared with Decoder
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=num_layers, dropout=dropout, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, out_dim or hidden_size)

        self.pos_encode = PositionalEncoding(out_dim or hidden_size, dropout, scale=time_scale)

    def forward(self, x, *ignore, skip_pos_encode=False):
        x = x.permute(0, 2, 1).contiguous()
        x = self.batchnorm(x)
        x = x.permute(0, 2, 1).contiguous()
        x, hidden = self.gru(x)
        # remove bi directional artifacts
        hidden = hidden[:2,:,:] + hidden[2:,:,:]
        x = self.fc(x)
        # x = F.tanh(x) # # # # # # #  # # # #  # # # # #  # # #  # # # # #  # # # # # TODO
        # x = torch.log1p(F.relu(x))
        if not skip_pos_encode:
            x = self.pos_encode(x)
        return x, hidden

# Duration: multy context attention


In [ ]:
class MultyContextAttention(nn.Module):
    def __init__(self, embedding_size, embedding_audio_size, hidden_size, vocab_size, device, dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size

        self.encoder = Encoder(hidden_size=embedding_size, embedding_size=embedding_size, num_layers=2, dropout=dropout, time_scale=None)

        self.encoder_transcription = Encoder(hidden_size=hidden_size, embedding_size=embedding_size, num_layers=2, dropout=dropout, time_scale=None)
        self.decoder_transcription = Decoder(embedding_size, hidden_size, hidden_size, num_layers=2, dropout=dropout, time_scale=None)

        self.encoder_audio = Encoder(hidden_size=hidden_size, embedding_size=embedding_audio_size, num_layers=2, dropout=dropout, time_scale=None)
        self.decoder_audio = Decoder(embedding_size, hidden_size, hidden_size, num_layers=2, dropout=dropout, time_scale=None)

        self.direct = nn.Linear(embedding_size, hidden_size)
        self.fast = nn.Linear(hidden_size * 2, hidden_size)
        self.chain = nn.Linear(hidden_size, embedding_size)
        self.shuffle = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)

        self.device = device
        self.mode = "fast"

        load(self, "/content/spn/trained_weights/duration-final-[10-25].pth")
        # load(self,  "/content/spn/trained_weights/duration-final-[5-25].pth")

    def forward(self, input_sequence, mask, features_audio, masks_audio):
        batch_size, out_seq_len, *features = input_sequence.shape

        encoded_inputs, hidden = self.encoder(input_sequence)

        if self.mode not in ['direct', 'audio']:
            encoder_transcription_outputs, hidden_transcription = self.encoder_transcription(input_sequence)

        if self.mode not in ['direct', 'trans']:
            encoder_audio_outputs, hidden_audio = self.encoder_audio(features_audio)

        if self.mode == "direct":
            output = self.direct(encoded_inputs)
        elif self.mode == "audio":
            output, hidden_audio = self.decoder_audio(encoded_inputs, hidden_audio, encoder_audio_outputs, masks_audio)
        elif self.mode == "trans":
            output, hidden_transcription = self.decoder_transcription(encoded_inputs, hidden_transcription, encoder_transcription_outputs, mask)

        elif self.mode == "fast":
            output_transcription, hidden_transcription = self.decoder_transcription(encoded_inputs, mask, hidden_transcription, encoder_transcription_outputs, mask)
            output_audio, hidden_audio = self.decoder_audio(encoded_inputs, mask, hidden_audio, encoder_audio_outputs, masks_audio)
            output = torch.cat((output_transcription, output_audio), 2)
            output = self.fast(output)

        elif self.mode == "chain":
            output_audio, hidden_audio = self.decoder_audio(encoded_inputs, hidden_audio, encoder_audio_outputs, masks_audio)
            # mangle the output to be acceptable for one more pass through the encoders
            output_audio, hidden_audio = self.chain(output_audio), hidden_audio + hidden_transcription
            # pass hidden audio as a hint
            output, hidden_transcription = self.decoder_transcription(output_audio, hidden_audio, encoder_transcription_outputs, mask)

        elif self.mode == "iter":
            output = torch.zeros(batch_size, out_seq_len, self.vocab_size).to(self.device) # tensor to store decoder outputs
            for t in range(out_seq_len):
                decoder_input = encoded_inputs[:,t:(t+1),:]

                output_transcription, hidden_transcription = self.decoder_transcription(decoder_input, hidden_transcription, encoder_transcription_outputs, mask)
                output_audio, hidden_audio = self.decoder_audio(decoder_input, hidden_audio, encoder_audio_outputs, masks_audio)

                out = torch.cat((output_transcription, output_audio), 2)
                output[:,t:(t+1),:] = self.out(out)
            return F.relu(output)

        else:
            raise Exception("wrong mode")

        output = torch.log1p(F.relu(output))
        output = self.shuffle(output)
        output = F.relu(output)
        output = self.out(output)
        output = F.relu(output)

        total_duration = masks_audio.sum(1) * ms_per_step if masks_audio is not None else features_audio.shape[1] * ms_per_step
        if mask is not None:
            # print(output.shape, mask.shape)
            # output = torch.bmm(output, mask.unsqueeze(2).float())
            output = (output * mask.unsqueeze(2).float())

        current_dur = (output.sum(1) * DURATION_SCALER).squeeze(1)
        # print(total_duration.shape, current_dur.shape)

        current_duration_scalers = total_duration / (current_dur)

        # print(current_duration_scalers.shape, current_duration_scalers)
        output = (output.squeeze(2) * current_duration_scalers.unsqueeze(1)).unsqueeze(2)
        # print(output.shape)
        # print((output.sum(1) * DURATION_SCALER).squeeze(1))
        # print(total_duration.shape, total_duration)
        # print(output.sum(1).shape, )
        # output = output * (total_duration / output.sum(1))
        # raise Exception
        return output

duration_combined_model = MultyContextAttention(KNOWN_LABELS_COUNT, INPUT_SIZE, 256, 1, device).to(device).eval()
evaluate(duration_combined_model, train_eval_dataset.batch(64), loss_function=MaskedLoss(), train_function=duration_trainer)
show_duration_og(duration_combined_model, test_dataset, sample_size=2000)
torch.cuda.empty_cache()
#  Evaluation on train - avg_loss: 0.0064 count:58 Total loss:0.3695
"""
[duration scaled] DIFF abs mean: 22.21ms (14.83) min:0.00ms max:3443.59ms
	38.2%	 < 10ms
	63.0%	 < 20ms
	77.4%	 < 30ms
	85.6%	 < 40ms
	90.4%	 < 50ms
	94.4%	 < 65ms
	97.0%	 < 85ms
	98.4%	 < 110ms
	99.2%	 < 140ms

[duration] DIFF abs mean: 22.15ms (14.53) min:0.00ms max:3456.12ms
	20.6%	 < 5ms		38.7%	 < 10ms
	52.6%	 < 15ms		63.5%	 < 20ms
	71.8%	 < 25ms		78.0%	 < 30ms
	82.5%	 < 35ms		86.2%	 < 40ms
	88.9%	 < 45ms		91.0%	 < 50ms
	92.6%	 < 55ms		93.8%	 < 60ms
	94.8%	 < 65ms		95.7%	 < 70ms
	96.3%	 < 75ms		96.8%	 < 80ms
	97.3%	 < 85ms		97.7%	 < 90ms
	97.9%	 < 95ms		98.1%	 < 100ms
	98.3%	 < 105ms		100.0%	 < 9999ms
20.55% 38.66% 52.65% 63.49% 71.78% 78.00% 82.47% 86.16% 88.88% 90.98% 92.59% 93.85% 94.85% 95.68% 96.29% 96.81% 97.32% 97.66% 97.89% 98.09%
[position] DIFF abs mean: 284.06ms (276.45) min:0.01ms max:4280.40ms
	2.7%	 < 5ms		5.2%	 < 10ms
	7.6%	 < 15ms		9.7%	 < 20ms
	11.8%	 < 25ms		13.6%	 < 30ms
	15.5%	 < 35ms		17.2%	 < 40ms
	18.7%	 < 45ms		20.3%	 < 50ms
	21.7%	 < 55ms		23.1%	 < 60ms
	24.5%	 < 65ms		25.8%	 < 70ms
	27.2%	 < 75ms		28.4%	 < 80ms
	29.6%	 < 85ms		30.8%	 < 90ms
	31.9%	 < 95ms		33.0%	 < 100ms
	34.0%	 < 105ms		100.0%	 < 9999ms
2.68% 5.23% 7.58% 9.69% 11.78% 13.65% 15.46% 17.16% 18.74% 20.29% 21.74% 23.13% 24.50% 25.82% 27.18% 28.41% 29.58% 30.80% 31.88% 33.02%

=====  AFTER SCALING
[duration] DIFF abs mean: 19.59ms (0.00) min:0.00ms max:1616.81ms
	20.4%	 < 5ms		39.1%	 < 10ms
	54.5%	 < 15ms		66.5%	 < 20ms
	75.4%	 < 25ms		81.9%	 < 30ms
	86.4%	 < 35ms		89.8%	 < 40ms
	92.1%	 < 45ms		93.7%	 < 50ms
	95.0%	 < 55ms		96.0%	 < 60ms
	96.7%	 < 65ms		97.2%	 < 70ms
	97.7%	 < 75ms		98.0%	 < 80ms
	98.3%	 < 85ms		98.5%	 < 90ms
	98.7%	 < 95ms		98.8%	 < 100ms
	98.9%	 < 105ms		100.0%	 < 9999ms
20.37% 39.06% 54.51% 66.53% 75.40% 81.89% 86.43% 89.76% 92.07% 93.74% 95.00% 96.00% 96.69% 97.24% 97.65% 97.98% 98.27% 98.47% 98.67% 98.82%
[position] DIFF abs mean: 71.81ms (-10.80) min:0.00ms max:1616.81ms
	8.1%	 < 5ms		13.3%	 < 10ms
	18.5%	 < 15ms		23.4%	 < 20ms
	28.5%	 < 25ms		33.1%	 < 30ms
	37.7%	 < 35ms		42.1%	 < 40ms
	46.2%	 < 45ms		50.0%	 < 50ms
	53.7%	 < 55ms		57.1%	 < 60ms
	60.2%	 < 65ms		63.1%	 < 70ms
	65.8%	 < 75ms		68.3%	 < 80ms
	70.7%	 < 85ms		73.0%	 < 90ms
	75.2%	 < 95ms		77.0%	 < 100ms
	78.8%	 < 105ms		100.0%	 < 9999ms
8.06% 13.26% 18.49% 23.43% 28.47% 33.12% 37.68% 42.06% 46.24% 50.02% 53.69% 57.05% 60.17% 63.12% 65.76% 68.33% 70.73% 73.04% 75.16% 76.99%
""";

In [ ]:
# loss_function = MaskedSoftL1(5)
# loss_function = MaskedMSE()
loss_function = MaskedLoss()
# loss_function = DivMaskedMSE(5)

train_batch_size = 64
if WIN_STEP < 0.010:
    train_batch_size = 16

train(duration_combined_model, 4, train_dataset.batch(train_batch_size), [train_eval_dataset.batch(64)], loss_function=loss_function, train_function=duration_trainer, lr_decay=0.98, lr=0.000015, weight_decay=1e-05)

# export_model(duration_combined_model, "/content/drive/My Drive/dataset/multy-attention-duration-normalized-3-[10-25].pth")
show_duration_og(duration_combined_model, test_dataset, sample_size=2000)
duration_combined_model.eval()
torch.cuda.empty_cache()

In [ ]:
# draw_duration(duration_combined_model, test_dataset, 36)
# export_model(duration_combined_model, "/content/drive/My Drive/dataset/multy-attention-duration-final-[5-25].pth")


# POS = new

In [ ]:
class PositionSimple(nn.Module):
    class Mode:
        weights = "weights"
        position = "position"
        gradient = "gradient"
        argmax = "argmax"

    def __getattr__(self, item):
        cleaned = item.replace("is_", "").replace("with_", "")
        if hasattr(self.Mode, cleaned):
            mode = getattr(self.Mode, cleaned)
            if "is_" in item:
                return self.mode == mode
            elif "with_" in item:
                self.mode = mode
                return self
        return super().__getattr__(item)

    def __dir__(self):
        return super().__dir__() + [f"is_{k}" for k in self.Mode.keys] + [f"with_{k}" for k in self.Mode.keys]

    def __init__(self, embedding_size, embedding_audio_size, hidden_size, vocab_size, device, attention_size=None, dropout=0.35):
        super().__init__()
        out_dim = hidden_size # vocab_size
        self.encoder_transcription = Encoder(hidden_size, embedding_size, out_dim=out_dim, num_layers=2, dropout=dropout, time_scale=POS_TRANSCRIPTION_SCALE)

        self.encoder_audio = Encoder(hidden_size, embedding_audio_size, out_dim=out_dim, num_layers=2, dropout=dropout, time_scale=POS_SCALE)

        # self.encoder_transcription_2 = Encoder(hidden_size, out_dim, out_dim=vocab_size, num_layers=2, dropout=dropout, time_scale=None)
        # self.encoder_audio_2 = Encoder(hidden_size, out_dim, out_dim=vocab_size, num_layers=2, dropout=dropout, time_scale=POS_SCALE)

        self.attn = Attention(None)
        self.gradient = (torch.cumsum(torch.ones(2 ** 14), 0).unsqueeze(1) - 1).cuda()
        self.zero = torch.zeros(256, 2048, vocab_size).to(device)
        self.pos_encode = PositionalEncoding(vocab_size, dropout, scale=POS_SCALE)

        print("scale:", POS_TRANSCRIPTION_SCALE)
        self.vocab_size = vocab_size
        self.device = device
        self.to(device)
        self.mode = self.Mode.gradient
        self.flags = {}
        self.use_iter = True
        self.use_pos_encode = True

        self.t_transformer = nn.Sequential(
            nn.Linear(embedding_size, 32),
            nn.Sigmoid(),
            nn.Linear(32, embedding_size),
            nn.Sigmoid()
        ).to(device)

        self.a_transformer = nn.Sequential(
            nn.Linear(embedding_audio_size, 32),
            nn.Sigmoid(),
            nn.Linear(32, embedding_audio_size),
            nn.Sigmoid()
        ).to(device)


    def weights_to_positions(self, weights, argmax=False):
        batch_size, audio_size, input_size = weights.shape

        batch, trans_len, seq_len = weights.shape
        if argmax:
            return weights.max(2)[1][:,:-1]
        positions = (self.gradient[:seq_len] * weights.transpose(1, 2)).sum(1)[:,:-1]
        return positions

    def forward(self, features_transcription, mask_transcription, features_audio, mask_audio):
        features_transcription = features_transcription.clone()
        features_transcription[:,:-1] += features_transcription[:,1:] * 0.55

        features_transcription = self.t_transformer(features_transcription)
        features_audio = self.a_transformer(features_audio)

        encoder_transcription_outputs, _ = self.encoder_transcription(features_transcription, skip_pos_encode=not self.use_pos_encode) # # # #

        encoder_audio_outputs, _ = self.encoder_audio(features_audio, skip_pos_encode=not self.use_pos_encode)


        # w = self.attn(F.tanh(encoder_transcription_outputs), mask_transcription, F.tanh(encoder_audio_outputs), mask_audio)

        if "iter" and True:
            encoder_transcription_outputs = F.relu(encoder_transcription_outputs)
            encoder_audio_outputs = F.relu(encoder_audio_outputs)
            # tensor to store decoder outputs
            batch_size, out_seq_len, _ = features_transcription.shape
            w = torch.zeros(batch_size, out_seq_len, features_audio.shape[1]).to(self.device)

            w_masks, w_mask, iter_mask_audio = [], None, mask_audio
            for t in range(out_seq_len):
                iter_input = encoder_transcription_outputs[:,t:(t+1),:]
                iter_memory = encoder_audio_outputs

                if len(w_masks) > 1:
                    w_mask = w_masks[0]
                    w_mask_b = w_masks[1]

                    w_mask = torch.clamp(w_mask, min=0.0, max=1)
                    w_mask[w_mask < 0.1] = 0
                    w_mask[w_mask > 0.1] = 1

                    w_mask_b = torch.clamp(w_mask_b, min=0.0, max=1)
                    w_mask_b[w_mask_b < 0.1] = 0

                    pad = 0.00
                    a, b = torch.split(iter_memory, 128, dim=2)
                    a = a * (w_mask.unsqueeze(2) * (1 - pad) + pad)
                    b = b * (w_mask_b.unsqueeze(2) * (1 - pad) + pad)
                    iter_memory = torch.cat([a, b], dim=2)
                    iter_mask_audio = mask_audio * ( w_mask > 0.1) if mask_audio is not None else w_mask > 0.1

                    # iter_memory = iter_memory * (w_mask.unsqueeze(2) * (1 - pad) + pad)
                # print(iter_input.shape, mask_transcription.shape, (iter_memory).shape, iter_mask_audio.shape)
                iter_mask_transcription = mask_transcription[:,t:(t+1)] if mask_transcription is not None else None
                w_slice = self.attn(iter_input, iter_mask_transcription, (iter_memory), iter_mask_audio)

                if w_mask is not None:
                    w[:,t:(t+1), :] = w_slice * w_mask.unsqueeze(1)
                else:
                    w[:,t:(t+1), :] = w_slice

                w_mask = w_slice.squeeze(1).clone()
                w_mask = torch.cumsum(w_mask, dim=1).detach()
                w_masks.append(w_mask)
                w_masks = w_masks[-2:]


        # elif self.use_iter:
        #     encoder_transcription_outputs = F.tanh(encoder_transcription_outputs)
        #     encoder_audio_outputs = F.tanh(encoder_audio_outputs)
        #     # tensor to store decoder outputs
        #     batch_size, out_seq_len, _ = features_transcription.shape
        #     w = torch.zeros(batch_size, out_seq_len, features_audio.shape[1]).to(self.device)

        #     w_masks, w_mask, iter_mask_audio = [], None, mask_audio
        #     for t in range(out_seq_len):
        #         iter_input = encoder_transcription_outputs[:,t:(t+1),:]
        #         iter_memory = encoder_audio_outputs

        #         if len(w_masks) > 1:
        #             w_mask = w_masks[0]
        #             w_mask_b = w_masks[1]

        #             # w_mask = torch.clamp(w_mask, min=0.0, max=1)
        #             # w_mask[w_mask < 0.45] = 0
        #             w_mask[w_mask > 0.1] = 1
        #             # TODO: combine masks of previos two

        #             # w_mask_b = torch.clamp(w_mask_b, min=0.0, max=1)
        #             # w_mask_b[w_mask_b < 0.1] = 0

        #             pad = 0.0
        #             # a, b = torch.split(iter_memory, 128, dim=2)
        #             # a = a * (w_mask.unsqueeze(2) * (1 - pad) + pad)
        #             # b = b * (w_mask_b.unsqueeze(2) * (1 - pad) + pad)
        #             # iter_memory = torch.cat([a, b], dim=2)
        #             low = 0.1
        #             iter_mask_audio = mask_audio * (w_mask > low) if mask_audio is not None else w_mask > low

        #             iter_memory = iter_memory * (w_mask.unsqueeze(2) * (1 - pad) + pad)

        #         w_slice = self.attn(iter_input, mask_transcription[:,t:(t+1)], iter_memory, iter_mask_audio)

        #         if w_mask is not None and 0:
        #             w[:,t:(t+1), :] = w_slice * w_mask.unsqueeze(1)
        #         else:
        #             w[:,t:(t+1), :] = w_slice

        #         w_mask = w_slice.squeeze(1).clone()
        #         w_mask = torch.cumsum(w_mask, dim=1).detach()
        #         w_masks.append(w_mask)
        #         w_masks = w_masks[-2:]

        if self.is_weights:
            return w

        if self.is_gradient or self.is_argmax:
            return self.weights_to_positions(w, argmax=self.is_argmax)

        batch, seq_len, dimensions = encoder_audio_outputs.shape
        processsed_audio = self.zero[:batch,:seq_len,:self.vocab_size]
        pos = self.pos_encode(processsed_audio)
        position_encodes = torch.bmm(w, pos)

        if self.is_position:
            return position_encodes[:,:-1]


position_model = PositionSimple(KNOWN_LABELS_COUNT, INPUT_SIZE, 256, POS_DIM, device).to(device).with_gradient
load(position_model, "/content/spn/trained_weights/position_model-final.pth")

"""
TOTAL 344903.16
[position] DIFF abs mean: 7.01ms (-0.39) min:0.00ms max:780.91ms
	55.97%	 < 5ms		81.93%	 < 10ms
	90.88%	 < 15ms		94.61%	 < 20ms
	96.62%	 < 25ms		97.79%	 < 30ms
	98.48%	 < 35ms		98.87%	 < 40ms
	99.15%	 < 45ms		99.35%	 < 50ms
	99.48%	 < 55ms		99.57%	 < 60ms
	99.65%	 < 65ms		99.67%	 < 70ms
	99.70%	 < 75ms		99.72%	 < 80ms
	99.74%	 < 85ms		99.76%	 < 90ms
	99.77%	 < 95ms		99.79%	 < 100ms
	99.81%	 < 105ms		100.01%	 < 9999ms
55.97% 81.93% 90.88% 94.61% 96.62% 97.79% 98.48% 98.87% 99.15% 99.35% 99.48% 99.57% 99.65% 99.67% 99.70% 99.72% 99.74% 99.76% 99.77% 99.79%
"""

def eval_border_agreement(duration_combined_model=None):
    show_position_batched(position_model.with_gradient, test_dataset, duration_combined_model=duration_combined_model)
    show_position_batched(position_model.with_gradient, train_dataset, duration_combined_model=duration_combined_model)
    if duration_combined_model is not None:
        print(" -  -  -  -  -  - WITHOUT -  -  -  -  -  - ")
        show_position_batched(position_model.with_gradient, test_dataset)
        show_position_batched(position_model.with_gradient, train_dataset)

# evaluate(position_model, train_dataset.batch(64), loss_function=MaskedMSE(), train_function=position_gradient_trainer) # 0.15
# evaluate(position_model, test_dataset.batch(64), loss_function=MaskedMSE(), train_function=position_gradient_trainer) # 2.1
eval_border_agreement(duration_combined_model=duration_combined_model)
torch.cuda.empty_cache()
!nvidia-smi

## Training

In [ ]:
torch.cuda.empty_cache()

class DivMaskedMSE(nn.Module):
    mse = nn.MSELoss()
    l1 = nn.L1Loss()
    def __init__(self, cutoff, flip=False):
        super().__init__()
        self.cutoff = cutoff
        self.flip = flip
        print("CUTOFF", cutoff)

    def forward(self, pred, target, mask, weight):
        # print(weight.shape)
        diff = torch.abs(pred - target)
        if not self.flip:
            diff = diff > (self.cutoff or random.randint(0, 3))
        else:
            diff = diff < (self.cutoff or random.randint(0, 3))

        pred = pred * weight[:,:-1]
        target = target * weight[:,:-1]

        mask_diff = mask & diff
        pred = torch.mul(pred, mask_diff)
        target = torch.mul(target, mask_diff)
        mse = self.mse(pred, target)
        # return mse

        mask_diff = mask & ~diff
        pred = torch.mul(pred, mask_diff)
        target = torch.mul(target, mask_diff)
        l1 = self.l1(pred, target)

        return mse + l1

class MaskedL1(nn.Module):
    l1 = nn.L1Loss()

    def forward(self, pred, target, mask):
        pred = torch.mul(pred, mask)
        target = torch.mul(target, mask)
        l1 = self.l1(pred, target)

        return l1


class MaskedThing(nn.Module):
    l1 = nn.L1Loss()
    mse = nn.MSELoss()

    def forward(self, pred, target, mask):
        pred = torch.log1p(F.relu(pred))
        target = torch.log1p(F.relu(target))
        pred = torch.mul(pred, mask)
        target = torch.mul(target, mask)
        l1 = self.mse(pred, target)

        return l1

# toy_dataset = DirectMaskDataset(train_files, limit=2043)
work_dataset = train_dataset
# work_dataset = train_augment_dataset

evaluation = [toy_dataset.batch(64)]
evaluation = [test_dataset.batch(64), train_eval_dataset.batch(64)]

train_batch_size = 32
if WIN_STEP < 0.010:
    train_batch_size = 16

f = .35
n = 1
# eval_border_agreement(duration_combined_model=duration_combined_model)

# train(position_model.with_gradient, int(10 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=MaskedMSE(), train_function=position_gradient_trainer, lr_decay=0.951, lr= f * 0.000161 * 0.131 * .10, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)

# train(position_model.with_position, int(8 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=CosineLoss(), train_function=position_encode_trainer, lr_decay=0.93, lr= f * 0.000051 * 0.005)
# eval_border_agreement(duration_combined_model=duration_combined_model)

# train(position_model.with_gradient, int(8 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=MaskedMSE(), train_function=position_gradient_trainer, lr_decay=0.991, lr= f * 0.000161 * 0.131 * 3.1 * 0.01, weight_decay=1e-05 * 14)

# export_model(position_model, "/content/drive/My Drive/dataset/position_model-pure-derp.pth")
# show_position_batched(position_model, toy_dataset, report_error=750)
# eval_border_agreement(duration_combined_model=duration_combined_model)

# NORMAL
# train(position_model.with_gradient, int(10 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=MaskedMSE(), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * .01, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)

# train(position_model.with_gradient, int(9 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(10, flip=True), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * .05, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)

# train(position_model.with_gradient, int(30 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(13, flip=False), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)

# train(position_model.with_gradient, int(30 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(5, flip=False), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)

# train(position_model.with_gradient, int(30 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(21, flip=False), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)


# torch.cuda.empty_cache()
# for i in range(10, 0, -1):
#     f = .25
#     # n= 0.8
#     load(position_model, "/content/drive/My Drive/dataset/position_model-final-3.pth")
#     train(position_model.with_gradient, int(5 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(i, flip=False), train_function=position_gradient_trainer, lr_decay=0.985, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
#     eval_border_agreement(duration_combined_model=duration_combined_model)

torch.cuda.empty_cache()
f = .023
n= 1.8
# load(position_model, "/content/drive/My Drive/dataset/position_model-final-3.pth")
train(position_model.with_gradient, int(5 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(1.5, flip=False), train_function=position_gradient_trainer, lr_decay=0.985, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
# train(position_model.with_gradient, int(5 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=MaskedMSE(), train_function=position_gradient_trainer, lr_decay=0.985, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
eval_border_agreement(duration_combined_model=None)
# export_model(position_model, "/content/drive/My Drive/dataset/position_model-derp.pth")

# SIMPLE
# train(position_model.with_gradient, int(30 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=MaskedMSE(), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * 3.1, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)
# train(position_model.with_gradient, int(10 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(5), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * .1, weight_decay=1e-05 * 14)
# eval_border_agreement(duration_combined_model=duration_combined_model)
# train(position_model.with_gradient, int(10 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(5), train_function=position_gradient_trainer, lr_decay=0.98, lr= f * 0.000161 * 0.131 * .1, weight_decay=1e-05 * 14)
# train(position_model.with_position, int(30 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=CosineLoss(), train_function=position_encode_trainer, lr_decay=0.93, lr= f * 0.000051 * 0.03)
# eval_border_agreement(duration_combined_model=duration_combined_model)

torch.cuda.empty_cache()

# 49.97% 77.86% 89.11% 93.87% 96.27% 97.61% 98.35% 98.84% 99.13% 99.33% 99.46% 99.57% 99.61% 99.69% 99.73% 99.75% 99.77% 99.78% 99.79% 99.81%
# 56.05% 81.85% 90.93% 94.65% 96.66% 97.77% 98.47% 98.88% 99.16% 99.34% 99.49% 99.54% 99.62% 99.65% 99.68% 99.70% 99.72% 99.74% 99.75% 99.77%
# % % % % % % % % % % % % % % % % % % % %
# 55.97% 81.93% 90.88% 94.61% 96.62% 97.79% 98.48% 98.87% 99.15% 99.35% 99.48% 99.57% 99.65% 99.67% 99.70% 99.72% 99.74% 99.76% 99.77% 99.79%

In [ ]:
train(position_model.with_gradient, int(5 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(1.5, flip=False), train_function=position_gradient_trainer, lr_decay=0.985, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
# train(position_model.with_gradient, int(5 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=MaskedMSE(), train_function=position_gradient_trainer, lr_decay=0.985, lr= f * 0.000161 * 0.131 * .2, weight_decay=1e-05 * 14)
eval_border_agreement(duration_combined_model=None)

In [ ]:
                                                            work_dataset = train_augment_dataset
train(position_model.with_gradient, int(10 * n), work_dataset.batch(train_batch_size), evaluation, loss_function=DivMaskedMSE(5, flip=False), train_function=position_gradient_trainer, lr_decay=0.985, lr= f * 0.000161 * 0.131 * .1, weight_decay=1e-05 * 14)
eval_border_agreement(duration_combined_model=duration_combined_model)

In [ ]:
# export_model(position_model, "/content/drive/My Drive/dataset/position_model-final-4.pth")

# eval_border_agreement(duration_combined_model=duration_combined_model)
# export_model(position_model, "/content/drive/My Drive/dataset/position_model-simple-final.pth")

### Simple model scores

In [ ]:
# [position] DIFF abs mean: 8.30ms (-2.00) min:0.00ms max:748.36ms
# 	57.3%	 < 5ms		82.1%	 < 10ms
# 	90.6%	 < 15ms		94.0%	 < 20ms
# 	95.8%	 < 25ms		96.8%	 < 30ms
# 	97.5%	 < 35ms		98.0%	 < 40ms
# 	98.3%	 < 45ms		98.5%	 < 50ms
# 	98.6%	 < 55ms		98.8%	 < 60ms
# 	98.8%	 < 65ms		98.9%	 < 70ms
# 	99.0%	 < 75ms		99.0%	 < 80ms
# 	99.0%	 < 85ms		99.1%	 < 90ms
# 	99.1%	 < 95ms		99.1%	 < 100ms
# 	99.2%	 < 105ms		100.0%	 < 9999ms
# 57.28% 82.07% 90.56% 94.00% 95.77% 96.85% 97.54% 97.99% 98.29% 98.50% 98.64% 98.77% 98.85% 98.91% 98.96% 99.01% 99.05% 99.08% 99.12% 99.15%
# dataset len 3196
# [position] DIFF abs mean: 5.75ms (-0.25) min:0.00ms max:313.08ms
# 	60.6%	 < 5ms		85.6%	 < 10ms
# 	93.6%	 < 15ms		96.7%	 < 20ms
# 	98.2%	 < 25ms		98.9%	 < 30ms
# 	99.3%	 < 35ms		99.6%	 < 40ms
# 	99.7%	 < 45ms		99.8%	 < 50ms
# 	99.8%	 < 55ms		99.8%	 < 60ms
# 	99.9%	 < 65ms		99.9%	 < 70ms
# 	99.9%	 < 75ms		99.9%	 < 80ms
# 	99.9%	 < 85ms		99.9%	 < 90ms
# 	99.9%	 < 95ms		99.9%	 < 100ms
# 	99.9%	 < 105ms		100.0%	 < 9999ms
# 60.61% 85.58% 93.64% 96.72% 98.17% 98.91% 99.34% 99.56% 99.69% 99.76% 99.81% 99.85% 99.87% 99.88% 99.89% 99.90% 99.91% 99.92% 99.92% 99.93%
#  -  -  -  -  -  - WITHOUT -  -  -  -  -  -
# dataset len 1344
# [id:173]  [  -18  2302]    32 True
# [position] DIFF abs mean: 7.32ms (0.32) min:0.00ms max:2302.34ms
# 	57.5%	 < 5ms		82.5%	 < 10ms
# 	91.0%	 < 15ms		94.5%	 < 20ms
# 	96.3%	 < 25ms		97.4%	 < 30ms
# 	98.1%	 < 35ms		98.5%	 < 40ms
# 	98.8%	 < 45ms		99.1%	 < 50ms
# 	99.2%	 < 55ms		99.3%	 < 60ms
# 	99.4%	 < 65ms		99.5%	 < 70ms
# 	99.5%	 < 75ms		99.6%	 < 80ms
# 	99.6%	 < 85ms		99.6%	 < 90ms
# 	99.6%	 < 95ms		99.7%	 < 100ms
# 	99.7%	 < 105ms		100.0%	 < 9999ms
# 57.55% 82.51% 91.05% 94.54% 96.32% 97.40% 98.08% 98.55% 98.84% 99.06% 99.21% 99.33% 99.42% 99.48% 99.53% 99.56% 99.59% 99.60% 99.62% 99.65%
# dataset len 3196
# [id:953]  [  -24   802]    49 True
# [id:2217]  [  -14   768]    64 True
# [id:2836]  [  -10   784]    36 True
# [position] DIFF abs mean: 6.02ms (0.35) min:0.00ms max:801.98ms
# 	60.6%	 < 5ms		85.6%	 < 10ms
# 	93.6%	 < 15ms		96.7%	 < 20ms
# 	98.1%	 < 25ms		98.9%	 < 30ms
# 	99.3%	 < 35ms		99.5%	 < 40ms
# 	99.7%	 < 45ms		99.7%	 < 50ms
# 	99.8%	 < 55ms		99.8%	 < 60ms
# 	99.8%	 < 65ms		99.8%	 < 70ms
# 	99.9%	 < 75ms		99.9%	 < 80ms
# 	99.9%	 < 85ms		99.9%	 < 90ms
# 	99.9%	 < 95ms		99.9%	 < 100ms
# 	99.9%	 < 105ms		100.0%	 < 9999ms
# 60.60% 85.56% 93.61% 96.69% 98.15% 98.89% 99.32% 99.54% 99.67% 99.74% 99.79% 99.82% 99.84% 99.85% 99.86% 99.87% 99.87% 99.87% 99.87% 99.87%

In [ ]:
# OTHER ?????
# scale: 8.344777745411855
#   Evaluation[] - avg_loss: 0.3700718 count:50 Total loss:18.5035897
#   Evaluation[] - avg_loss: 3.0839038 count:21 Total loss:64.7619789
# dataset len 1344
# [id:696]  [ -922    12]    53 True
# [id:848]  [-1497    79]    58 True
# [id:202]  [ -813    13]    51 True
# [id:933]  [  -13   760]    33 False
# [position] DIFF abs mean: 8.85ms (-2.27) min:0.00ms max:1496.71ms
# 	56.2%	 < 5ms		81.8%	 < 10ms
# 	90.7%	 < 15ms		94.4%	 < 20ms
# 	96.3%	 < 25ms		97.5%	 < 30ms
# 	98.1%	 < 35ms		98.5%	 < 40ms
# 	98.8%	 < 45ms		99.0%	 < 50ms
# 	99.1%	 < 55ms		99.2%	 < 60ms
# 	99.3%	 < 65ms		99.3%	 < 70ms
# 	99.3%	 < 75ms		99.4%	 < 80ms
# 	99.4%	 < 85ms		99.4%	 < 90ms
# 	99.4%	 < 95ms		99.4%	 < 100ms
# 	99.4%	 < 105ms		100.0%	 < 9999ms
# 56.17% 81.83% 90.74% 94.36% 96.32% 97.46% 98.13% 98.55% 98.80% 98.97% 99.11% 99.19% 99.26% 99.30% 99.33% 99.36% 99.38% 99.40% 99.41% 99.43%
# dataset len 3196
# [position] DIFF abs mean: 5.34ms (0.13) min:0.00ms max:550.61ms
# 	60.4%	 < 5ms		86.6%	 < 10ms
# 	95.2%	 < 15ms		98.2%	 < 20ms
# 	99.4%	 < 25ms		99.8%	 < 30ms
# 	99.9%	 < 35ms		100.0%	 < 40ms
# 	100.0%	 < 45ms		100.0%	 < 50ms
# 	100.0%	 < 55ms		100.0%	 < 60ms
# 	100.0%	 < 65ms		100.0%	 < 70ms
# 	100.0%	 < 75ms		100.0%	 < 80ms
# 	100.0%	 < 85ms		100.0%	 < 90ms
# 	100.0%	 < 95ms		100.0%	 < 100ms
# 	100.0%	 < 105ms		100.0%	 < 9999ms
# 60.38% 86.56% 95.19% 98.19% 99.40% 99.80% 99.91% 99.95% 99.96% 99.96% 99.96% 99.96% 99.96% 99.96% 99.96% 99.96% 99.96% 99.96% 99.96% 99.96%
#  -  -  -  -  -  - WITHOUT -  -  -  -  -  -
# dataset len 1344
# [id:696]  [ -922    23]    53 True
# [id:173]  [  -13  2300]    32 True
# [id:848]  [-1160    79]    58 True
# [id:202]  [ -813   379]    51 True
# [id:933]  [  -13   760]    33 False
# [position] DIFF abs mean: 7.80ms (-0.17) min:0.00ms max:2299.52ms
# 	56.3%	 < 5ms		82.0%	 < 10ms
# 	90.9%	 < 15ms		94.5%	 < 20ms
# 	96.5%	 < 25ms		97.6%	 < 30ms
# 	98.3%	 < 35ms		98.7%	 < 40ms
# 	99.0%	 < 45ms		99.1%	 < 50ms
# 	99.3%	 < 55ms		99.3%	 < 60ms
# 	99.4%	 < 65ms		99.5%	 < 70ms
# 	99.5%	 < 75ms		99.5%	 < 80ms
# 	99.5%	 < 85ms		99.5%	 < 90ms
# 	99.6%	 < 95ms		99.6%	 < 100ms
# 	99.6%	 < 105ms		100.0%	 < 9999ms
# 56.27% 81.96% 90.87% 94.49% 96.46% 97.61% 98.28% 98.70% 98.96% 99.13% 99.27% 99.34% 99.41% 99.45% 99.48% 99.50% 99.52% 99.54% 99.55% 99.57%
# dataset len 3196
# [id:953]  [  -22   803]    49 True
# [id:2217]  [  -19   768]    64 True
# [id:2836]  [  -13   788]    36 True
# [position] DIFF abs mean: 5.65ms (0.68) min:0.00ms max:802.68ms
# 	60.3%	 < 5ms		86.5%	 < 10ms
# 	95.1%	 < 15ms		98.1%	 < 20ms
# 	99.3%	 < 25ms		99.7%	 < 30ms
# 	99.8%	 < 35ms		99.8%	 < 40ms
# 	99.9%	 < 45ms		99.9%	 < 50ms
# 	99.9%	 < 55ms		99.9%	 < 60ms
# 	99.9%	 < 65ms		99.9%	 < 70ms
# 	99.9%	 < 75ms		99.9%	 < 80ms
# 	99.9%	 < 85ms		99.9%	 < 90ms
# 	99.9%	 < 95ms		99.9%	 < 100ms
# 	99.9%	 < 105ms		100.0%	 < 9999ms
# 60.33% 86.48% 95.09% 98.09% 99.30% 99.70% 99.81% 99.85% 99.86% 99.86% 99.86% 99.86% 99.86% 99.86% 99.86% 99.86% 99.86% 99.86% 99.86% 99.86%
# Sat May  9 02:44:37 2020

# Audio: prediction

In [ ]:
T = 50      # Input sequence length
C = 20      # Number of classes (including blank)
N = 16      # Batch size
S = 30      # Target sequence length of longest target in batch
S_min = 10  # Minimum target length, for demonstration purposes

# Initialize random batch of input vectors, for *size = (T,N,C)
input = torch.randn(T, N, C).log_softmax(2).detach().requires_grad_()

# Initialize random batch of targets (0 = blank, 1:C = classes)
target = torch.randint(low=1, high=C, size=(N, S), dtype=torch.long)

input_lengths = torch.full(size=(N,), fill_value=T, dtype=torch.long)
target_lengths = torch.randint(low=S_min, high=S, size=(N,), dtype=torch.long)

showTensor(target.detach())
showTensor(input.detach())

ctc_loss = nn.CTCLoss()
loss = ctc_loss(input, target, input_lengths, target_lengths)
loss.backward()

In [ ]:
class LightLSTM(nn.Module):
    def __init__(self, feature_dim, out_dim, dropout_prob=0.05, with_hidden=False):
        super().__init__()
        self.batchnorm = nn.BatchNorm1d(feature_dim)
        self.hidden_size = 128
        self.rnn = nn.LSTM(feature_dim, self.hidden_size, batch_first=True, bidirectional=True, num_layers=2, dropout=dropout_prob)
        self.fc = nn.Linear(self.hidden_size * 2, out_dim)
        self.with_hidden = with_hidden
        load(self, "/content/spn/trained_weights/CTC-lstm-final.pth")

    def forward(self, features, masks, features_audio, masks_audio):
        x = features_audio
        x = x.permute(0, 2, 1).contiguous()
        x = self.batchnorm(x)
        x = x.permute(0, 2, 1).contiguous()
        x, (hidden, _) = self.rnn(x)
        x = self.fc(x)
        if self.with_hidden:
            hidden = hidden[:2,:,:] + hidden[2:,:,:]
            return x, hidden
        return x



audio_lstm = LightLSTM(INPUT_SIZE, KNOWN_LABELS_COUNT).to(device)
show_audio(audio_lstm, test_dataset, "test", plot_only=True)

In [ ]:
# work_dataset = train_dataset
# evaluation = [test_dataset.batch(64), train_eval_dataset.batch(64)]
# train(audio_lstm, 8, work_dataset.batch(32), evaluation, loss_function=LabelSmoothingLossAudio(KNOWN_LABELS_COUNT), train_function=audio_detection_trainer, lr_decay=0.98, lr=0.000161 * 0.131 * .1)
# show_audio(audio_lstm, test_dataset, "test")
# export_model(audio_lstm, "/content/drive/My Drive/dataset/lstm-audio-final.pth")

# Audio detection

In [ ]:
class MultyContextAttentionAudio(nn.Module):
    def __init__(self, embedding_size, embedding_audio_size, hidden_size, vocab_size, device, dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size

        # self.encoder = Encoder(embedding_audio_size, embedding_audio_size, num_layers=2, dropout=dropout, time_scale=None)
        self.encoder = LightLSTM(embedding_audio_size, vocab_size, with_hidden=True).to(device)

        self.encoder_transcription = Encoder(hidden_size, embedding_size, num_layers=2, dropout=dropout, time_scale=None)
        self.decoder_transcription = Decoder(vocab_size, hidden_size, vocab_size, num_layers=2, dropout=dropout, time_scale=None)

        self.encoder_audio = Encoder(hidden_size, embedding_audio_size, num_layers=2, dropout=dropout, time_scale=1)
        self.decoder_audio = Decoder(vocab_size, hidden_size, hidden_size, num_layers=2, dropout=dropout, time_scale=1)

        self.out = nn.Linear(hidden_size * 2, vocab_size)
        self.out_single = nn.Linear(hidden_size, vocab_size)
        self.out_chain = nn.Linear(hidden_size, vocab_size)
        self.out_direct = nn.Linear(vocab_size, vocab_size)

        self.fix_hidden = nn.Linear(self.encoder.hidden_size, hidden_size)
        self.pos_encode = PositionalEncoding(vocab_size, dropout, scale=1)

        self.device = device

        self.mode = "chain"
        load(self, "/content/spn/trained_weights/CTC-final.pth")

    def forward(self, features_transcription, mask_transcription, features_audio, mask_audio):
        features_audio = features_audio * 32768.0
        batch_size, audio_len, *features = features_audio.shape
        batch_size, out_seq_len, *features = features_transcription.shape

        # if mask_transcription is None: mask_transcription = torch.ones(batch_size, out_seq_len).bool().to(device)
        # if mask_audio is None: mask_audio = torch.ones(batch_size, audio_len).bool().to(device)

        encoded_inputs, hidden = self.encoder(features_transcription, mask_transcription, features_audio, mask_audio)


        if self.mode == "direct":
            return encoded_inputs

        encoded_inputs = self.pos_encode(encoded_inputs)
        hidden = self.fix_hidden(hidden)
        encoder_transcription_outputs, hidden_transcription = self.encoder_transcription(features_transcription)

        if self.mode == "single":
            output_transcription, hidden_transcription = self.decoder_transcription(encoded_inputs, mask_audio, hidden_transcription + hidden, encoder_transcription_outputs, mask_transcription)
            # return self.out_single(output_transcription)
            return output_transcription

        encoder_audio_outputs, hidden_audio = self.encoder_audio(features_audio)

        if self.mode == "fast":
            output_transcription, hidden_transcription = self.decoder_transcription(encoded_inputs, hidden_transcription, encoder_transcription_outputs, mask_transcription)
            output_audio, hidden_audio = self.decoder_audio(encoded_inputs, hidden_audio, encoder_audio_outputs, mask_audio)
            output = torch.cat((output_transcription, output_audio), 2)
            return self.out(output)

        if self.mode == "chain":
            output_transcription, hidden_transcription = self.decoder_transcription(
                # previous, mask_trans, hidden_state, encoder_outputs, mask_audio
                encoded_inputs, mask_audio, hidden_transcription + hidden, encoder_transcription_outputs, mask_transcription,
            )
            # encoded_inputs = encoded_inputs + self.out_single(output_transcription)
            # hidden_audio = hidden_audio + hidden_transcription + hidden
            output_audio, hidden_audio = self.decoder_audio(output_transcription, mask_audio , hidden_audio, encoder_audio_outputs, mask_audio)
            return self.out_chain(output_audio)

        # # elif self.mode == "iter":
        # #     outputs = torch.zeros(batch_size, out_seq_len, self.vocab_size).to(self.device) # tensor to store decoder outputs
        # #     output = outputs[:,:1].clone()

        # #     for t in range(out_seq_len):
        # #         decoder_input = encoded_inputs[:,t:(t+1),:]

        # #         output_transcription, hidden_transcription = self.decoder_transcription(decoder_input, hidden_transcription, encoder_transcription_outputs, mask)
        # #         output_audio, hidden_audio = self.decoder_audio(decoder_input, hidden_audio, encoder_audio_outputs, masks_audio)

        # #         output = torch.cat((output_transcription, output_audio), 2)
        # #         output = self.out(output)
        # #         outputs[:,t:(t+1),:] = output
        # else:
        #     raise Exception("wrong mode")

        return outputs

multy_context_audio_model = MultyContextAttentionAudio(KNOWN_LABELS_COUNT, INPUT_SIZE, 256, KNOWN_LABELS_COUNT, device).to(device);
# toy_dataset = DirectMaskDataset('train', limit=127)
# train_iter = BucketIterator(train_dataset, batch_size=64, sort_key=lambda x: len(x.features), sort=False, shuffle=True, sort_within_batch=True)
# evaluate_audio("train", train_iter, multy_context_audio_model)
#  Evaluation on train - avg_loss: 0.158870 count:72 Total loss:11.438619658350945
show_audio(multy_context_audio_model, test_dataset, "test", plot_only=True)

In [ ]:
# duration_model = duration_combined_model
torch.cuda.empty_cache()
work_dataset = train_dataset
evaluation = [test_dataset.batch(64), train_eval_dataset.batch(64)]
from time import time
for i in range(1):
	start = time()
	train(multy_context_audio_model, 8, work_dataset.batch(16), evaluation, loss_function=LabelSmoothingLossAudio(KNOWN_LABELS_COUNT), train_function=audio_detection_trainer, lr_decay=0.98, lr=0.000161 * 0.131 * 3.6)
	print(time()-start)


In [ ]:
# evaluation = [test_dataset.batch(64), train_eval_dataset.batch(64)]
train(multy_context_audio_model, 4, train_dataset.batch(16), evaluation, loss_function=LabelSmoothingLossAudio(KNOWN_LABELS_COUNT), train_function=audio_detection_trainer, lr_decay=0.98, lr=0.000161 * 0.131 * 3.6)
show_audio(multy_context_audio_model, test_dataset, "test")
show_audio(multy_context_audio_model, test_dataset, "test", duration_model=duration_combined_model)

# [test]
# [duration model]
# danger: dtw_error 31.4% wrong idx:69
# - warped_result: (376, 54)
# - truth:(376, 54)
# danger: dtw_error 31.3% wrong idx:933
# - warped_result: (643, 54)
# - truth:(643, 54)
# [(233.0, 944), (251.75, 371), (340.0, 497), (390.0, 665), (715.0, 933)]
# cache_hits: 0
# [DETECTION+DTW]AVERAGE ERROR: 8.12% COUNT:1344
# [DETECTION]AVERAGE ERROR: 8.73% COUNT:1344
# [test] DIFF abs mean: 6.98ms (0.07) min:0.00ms max:715.00ms
# 	49.3%	 < 5ms		77.9%	 < 10ms
# 	89.0%	 < 15ms		93.6%	 < 20ms
# 	95.9%	 < 25ms		97.3%	 < 30ms
# 	98.1%	 < 35ms		98.7%	 < 40ms
# 	99.0%	 < 45ms		99.2%	 < 50ms
# 	99.4%	 < 55ms		99.5%	 < 60ms
# 	99.6%	 < 65ms		99.7%	 < 70ms
# 	99.7%	 < 75ms		99.8%	 < 80ms
# 	99.8%	 < 85ms		99.8%	 < 90ms
# 	99.8%	 < 95ms		99.9%	 < 100ms
# 	99.9%	 < 105ms		100.0%	 < 9999ms
# 49.30% 77.92% 89.03% 93.57% 95.87% 97.27% 98.11% 98.66% 99.03% 99.24% 99.41% 99.53% 99.60% 99.67% 99.71% 99.75% 99.78% 99.81% 99.83% 99.86%

In [ ]:
device

# PLOTS

## Intro

In [1]:
from matplotlib import gridspec
from matplotlib.ticker import FormatStrFormatter

def showTensor(aTensor, title=None, figsize=(12, 9), hide_x=False, labels=None, cmap=None, fig=True):
    # print(aTensor.shape, aTensor.sum())
    if len(aTensor.shape) > 2:
        aTensor = aTensor[0]

    if hasattr(aTensor, 'detach'): aTensor = aTensor.detach()
    if hasattr(aTensor, 'cpu'): aTensor = aTensor.cpu()
    if hasattr(aTensor, 'numpy'): aTensor = aTensor.numpy()

    fig and plt.figure(figsize=figsize)
    plt.imshow(aTensor.T, cmap=cmap)
    title and plt.title(title)
    hide_x and plt.xticks([])
    if labels:
        for x, y, label in labels:
            plt.annotate(f"{label}",xy=(x + 2, y), color='white')

    plt.show()


def duration_to_encoding(durations, pos_map):
    return pos_map[durations.long()]

def separation(num=5):
    print((" - " * 32 +  "\n") * num)

ms_per_step = WIN_STEP * 1000


NameError: name 'WIN_STEP' is not defined

In [ ]:
FOUND_LABELS = dict([
  ('h#', 12600), ('ix', 11587), ('s', 10114), ('iy', 9663), ('n', 9569), ('r', 9064),
  ('tcl', 8978), ('l', 8157), ('kcl', 7823), ('ih', 6760), ('dcl', 6585), ('k', 6488),
  ('t', 5899), ('m', 5429), ('ae', 5404), ('eh', 5293), ('z', 5046), ('ax', 4956), ('q', 4834),
  ('d', 4793), ('axr', 4790), ('w', 4379), ('aa', 4197), ('ao', 4096), ('dh', 3879),
  ('dx', 3649), ('pcl', 3609), ('p', 3545), ('ay', 3242), ('ah', 3185), ('f', 3128),
  ('ey', 3088), ('b', 3067), ('sh', 3034), ('gcl', 3031), ('ow', 2913), ('er', 2846),
  ('g', 2772), ('v', 2704), ('bcl', 2685), ('ux', 2488), ('y', 2349), ('epi', 2000),
  ('ng', 1744), ('jh', 1581), ('hv', 1523), ('pau', 1343), ('nx', 1331), ('hh', 1313),
  ('el', 1294), ('ch', 1081), ('th', 1018), ('en', 974), ('oy', 947), ('aw', 945),
  ('uh', 756), ('uw', 725), ('ax-h', 493), ('zh', 225), ('em', 171), ('eng', 43), ])

total_phonemes = sorted(FOUND_LABELS.items(), key=lambda x:-x[1])

for row in zip(total_phonemes[::3], total_phonemes[1::3], total_phonemes[2::3]):
    for i, (p, c) in enumerate(row):
        print(i and "\t\t" or "", f"{p}\t{c}", end="", sep="")
    print()
for i, (p, c) in enumerate(total_phonemes[-1:]):
    print(i and "\t\t" or "", f"{p}\t{c}", end="", sep="")
print("\n")

In [ ]:
list(sorted([(test_dataset[i].features.shape[0], i) for i in range(1300)]))[:10]
all_datasets = [train_dataset, train_eval_dataset, test_dataset]
for ds in all_datasets:print(len(ds))
audio_durations = np.array(sum([[inp.features.shape[0] * ms_per_step for inp in dataset] for dataset in all_datasets], []))

In [ ]:
def display_histogram(errors, name="", unit='ms', plotting=False):
    errors = errors.copy()
    hist, bins = np.histogram(abs(errors))
    hist = np.round(hist / len(errors) * 100, 1)
    hist = np.cumsum(hist)

    print(f"[{name}] DIFF abs mean: {abs(errors).mean():.2f}{unit} ({errors.mean():.2f}) min:{abs(errors).min():.2f}{unit} max:{abs(errors).max():.2f}{unit}")
    rows = list(zip(hist, bins, bins[1:]))
    for R  in zip(rows[::2], rows[1::2]):
        s = ""
        for h, b, e in R:
            s += f"\t{h:.1f}%\t < {e:.0f}{unit}\t"
        print(s)

    f, axarr = plt.subplots(1, 2, figsize=(10, 3))
    axarr[0].bar(range(len(bins)-1), np.round(hist).astype(int))
    axarr[0].set_xticklabels([f"{x:.0f}{unit}" for x in bins], fontdict=None, minor=False)
    axarr[0].yaxis.set_major_formatter(FormatStrFormatter('%d%%'))

    axarr[1].hist(errors, bins=15)
    axarr[1].xaxis.set_major_formatter(FormatStrFormatter(f'%d{unit}'))
    return [f, *axarr]

f, ax1, ax2 = display_histogram(audio_durations, plotting=True)
ax1.title.set_text("1. Audio durations less than X by percentage")
ax2.title.set_text("2. Audio duration occurence count")
ax1.set_xlabel('Audio duration', fontsize=12)
ax1.set_ylabel('Percentage', fontsize=12)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlabel('Audio duration', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)
ax2.tick_params(axis='both', which='both', labelsize=12)
f.tight_layout()


In [ ]:
audio_phoneme_count = np.array(sum([[inp.border.shape[0] for inp in dataset] for dataset in all_datasets], []))
print(sum(audio_phoneme_count))
f, ax1, ax2 = display_histogram(audio_phoneme_count, plotting=True, unit='')

ax1.title.set_text("1. Phoneme count less than X by percentage")
ax2.title.set_text("2. Phoneme occurence count")
ax1.set_xlabel('Phoneme count', fontsize=12)
ax1.set_ylabel('Percentage', fontsize=12)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlabel('Phoneme count', fontsize=12)
ax2.set_ylabel('Audio segment count', fontsize=12)
ax2.tick_params(axis='both', which='both', labelsize=12)
f.tight_layout()

In [ ]:
inps = sum([[inp for inp in dataset] for dataset in all_datasets], [])
phoneme_map = defaultdict(list)
for inp in inps:
    phoneme_ids = np.argmax(inp.in_transcription.cpu(), axis=1).tolist()
    phoneme_durs = (inp.out_duration.cpu()* DURATION_SCALER).tolist()
    for pid, pdur in zip(phoneme_ids, phoneme_durs):
        phoneme_map[pid].append(pdur)



mean_phoneme_dur = sorted([[KNOWN_LABELS[pid], np.mean(val) ] for pid, val in phoneme_map.items()], key=lambda x: x[1])
# mean_phoneme_dur = (zip(*mean_phoneme_dur))
# plt.figure(figsize=(24, 6))
# plt.bar(*mean_phoneme_dur)
print(len(mean_phoneme_dur))
for row in zip(mean_phoneme_dur[::3], mean_phoneme_dur[1::3], mean_phoneme_dur[2::3]):
    for i, (p, c) in enumerate(row):
        print(i and "\t\t" or "", f"{p}\t{c:.0f}ms", end="", sep="")
    print()
for i, (p, c) in enumerate(mean_phoneme_dur[-2:]):
    print(i and "\t\t" or "", f"{p}\t{c:.0f}ms", end="", sep="")
print("\n.")

In [ ]:
inp: Utterance = test_dataset[36]
audio_file = inp.label_file
audio = sf.read(audio_file)[0]
plt.figure()
plt.plot(audio, '-', );
plt.figure()
plt.plot(audio[100:132], '-', );

s, audio2 = wavfile.read(audio_file)
print(audio.min(), audio.max())
print(audio2.min(), audio2.max())
print((audio2[:3] / audio[:3]).mean())


fig = plt.figure(figsize=(9, 3), constrained_layout=True)
gs = gridspec.GridSpec(ncols=2, nrows=1)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])

ax1.plot(audio, '-', )
ax1.title.set_text("1. Audio waveform")
ax1.set_xlabel('Audio sample index', fontsize=13)
ax1.set_ylabel('Audio intensity', fontsize=13)
ax1.tick_params(axis='both', which='both', labelsize=13)

ax2.plot(audio, '-', )
ax2.title.set_text("2. Audio waveform magnified")
ax2.set_xlabel('Audio sample index', fontsize=13)
ax2.set_ylabel('Audio intensity', fontsize=13)
ax2.tick_params(axis='both', which='both', labelsize=13)
ax2.set_xlim([100, 131])
ax2.set_ylim([-0.0009, 0.0009])

fig.tight_layout()

In [ ]:
pos = PositionalEncoding(26, 0.0, scale=1)
pos_map = pos.pe.transpose(0, 1)[0]

inp: Utterance = test_dataset[36]

audio = inp.features
length = audio.shape[0]

label_vec = inp.label_vec
actual_borders = inp.border
pos_feature = pos_map[:length,:]
print(actual_borders.long())

combined = pos_feature + audio.cpu() / 2

highlighted = torch.zeros(length, 8)
highlighted[actual_borders.long()] = 1

plt.figure(figsize=(9, 4))
plt.xlabel('Audio frame index', fontsize=13)
plt.ylabel('Position encoding features', fontsize=13)
showTensor(pos_feature, 'Position encoding', fig=False)
separation()

plt.figure(figsize=(9, 4))
plt.xlabel('Audio frame index', fontsize=13)
plt.ylabel('Transformed audio features', fontsize=13)
showTensor(audio.cpu(), 'Transformed audio representation', fig=False)
print("We can see the border at 57")


showTensor(combined, 'Audio combined with position encodingss')

In [ ]:
labels_ids = np.argmax(inp.in_transcription.cpu(), axis=1).tolist()
labels = [KNOWN_LABELS[lab] for lab in labels_ids]
labels_pos = inp.border.long().cpu().tolist()
print(labels, len(labels))

def show_numbers(numbers, labels, title):
    plt.figure(figsize=(12, 3))
    plt.plot(numbers, '-', );
    for i,j in enumerate(numbers):
        plt.annotate(f"{labels[i]}\n{j:.0f}",xy=(i-1,j-1))
    plt.title(title)
    plt.show()

# showTensor(audio.cpu(), 'Audio')
# showTensor(label_vec.cpu(), 'Labels one-hot encoded', labels=zip(labels_pos, labels_ids, labels), figsize=(12, 6))
# show_numbers(inp.border.cpu().tolist(), labels, 'In order of occurence phoneme border position as index of audio segment');
# show_numbers((inp.out_duration * DURATION_SCALER).cpu().tolist(), labels, 'In order of occurence phoneme durations in milliseconds');

fig = plt.figure(figsize=(9, 11), constrained_layout=True)
gs = gridspec.GridSpec(ncols=1, nrows=4, height_ratios=[3, 6, 3, 3])

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)
ax3 = fig.add_subplot(gs[2])
ax4 = fig.add_subplot(gs[3], sharex=ax3)


ax1.imshow(audio.cpu().numpy().T, aspect='auto')
ax1.title.set_text("1. Audio")
ax1.set_xlabel('Audio frame index', fontsize=13)
ax1.set_ylabel('Features', fontsize=13)
ax1.tick_params(axis='both', which='both', labelsize=13)

ax2.imshow(label_vec.cpu().numpy().T, aspect='auto')
ax2.title.set_text("2. Labels one-hot encoded")
ax2.set_xlabel('Audio frame index', fontsize=13)
ax2.set_ylabel('One-hot encoded phonemes', fontsize=13)
ax2.tick_params(axis='both', which='both', labelsize=13)
for x, y, label in zip(labels_pos, labels_ids, labels):
    ax2.annotate(f"{label}",xy=(x + 2, y), color='white')
ax2.annotate(f"{label}",xy=(x - 3, y + 1), color='white')

numbers =inp.border.cpu().tolist()
ax3.plot(numbers, '-', );
for i,j in enumerate(numbers):
    ax3.annotate(f"{labels[i]}\n{j:.0f}",xy=(i,j))

ax3.title.set_text("3. In order of occurence phoneme border position as index of audio segment")
ax3.set_xlabel('Phoneme order number', fontsize=13)
ax3.set_ylabel('Audio index', fontsize=13)
ax3.tick_params(axis='both', which='both', labelsize=13)
ax3.set_ylim([0, 140])

numbers = (inp.out_duration * DURATION_SCALER).cpu().tolist()
ax4.plot(numbers, '-', );
for i,j in enumerate(numbers):
    ax4.annotate(f"{labels[i]}\n{j:.0f}",xy=(i,j))

ax4.title.set_text("4. In order of occurence phoneme durations in milliseconds")
ax4.set_xlabel('Phoneme order number', fontsize=13)
ax4.set_ylabel('Phoneme duration', fontsize=13)
ax4.tick_params(axis='both', which='both', labelsize=13)
ax4.yaxis.set_major_formatter(FormatStrFormatter('%dms'))
ax4.set_ylim([0, 200])
fig.tight_layout()

In [ ]:
vec = pos_feature[57].clone()
pos_feature_high = pos_feature.clone()
pos_feature_high[57] = 0
# showTensor(pos_feature_high, 'Position encoding with border position')
# print("Highlighted section on the positon encoding matchin the temporal positon. Extracted as a vector to the left.")

vec_noised = (torch.rand_like(vec) * 1 + vec) / 2

# showTensor(vec.unsqueeze(0), 'Vector', figsize=(12, 3), hide_x=True)
# showTensor(vec_noised.unsqueeze(0), 'Vector\'', figsize=(12, 3), hide_x=True)

attn = F.softmax(torch.bmm(vec.unsqueeze(0).unsqueeze(0), pos_feature.unsqueeze(0).transpose(1, 2)).transpose(1, 2).squeeze(0), dim=0)
attn_noised = F.softmax(torch.bmm(vec_noised.unsqueeze(0).unsqueeze(0), pos_feature.unsqueeze(0).transpose(1, 2)).transpose(1, 2).squeeze(0), dim=0)


# fig = plt.figure(figsize=(12, 6))
# gs = gridspec.GridSpec(ncols=1, nrows=2, height_ratios=[5, 1], hspace=0.01)

# ax1 = fig.add_subplot(gs[0])
# ax2 = fig.add_subplot(gs[1], sharex=ax1)
# # plt.setp(ax1.get_xticklabels(), visible=False)

# # plt.setp([ax1, ax2], title='Test')
# # fig.suptitle('An overall title', size=20)


# # fig, a =  plt.subplots(nrows=2, figsize=, gridspec_kw={'hspace': 0., 'wspace': 0.}, sharex=True)
# ax1.imshow(pos_feature_high.numpy().T)
# ax1.title.set_text("Position encoding with a single vector extracted")
# ax2.plot(attn.numpy(), label='Vector')
# ax2.title.set_text("The activation of the vector over the positions")
# ax2.plot(attn_noised.numpy(), label='Vector with added noise')
# ax2.title.set_text("The activation of the vector over the positions")
# ax2.legend()


fig = plt.figure(figsize=(9, 6), constrained_layout=True,)
gs = gridspec.GridSpec(ncols=2, nrows=2, width_ratios=[1, 10], height_ratios=[5, 5])

ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 1], sharex=ax1)
ax3 = fig.add_subplot(gs[0, 0])
ax4 = fig.add_subplot(gs[1, 0])

ax3.title.set_text("2. Vector[57]")
ax3.imshow(vec.unsqueeze(0).cpu().numpy().T)
ax3.set_xticks([57])

vec_noised = (torch.rand_like(vec) + vec) / 2
ax4.title.set_text("2. Vector[57]+noise")
ax4.imshow(vec_noised.unsqueeze(0).cpu().numpy().T)
ax4.set_xticks([57])

ax1.imshow(pos_feature_high.numpy().T, aspect='auto')
ax1.title.set_text("1. Position encoding with a single vector extracted at index 57")
ax1.set_xlabel('Audio frame index', fontsize=13)
ax1.set_ylabel('Position encoding features', fontsize=13)
ax1.tick_params(axis='both', which='both', labelsize=13)

ax2.title.set_text("3. The activation of the vector over the positions")
ax2.plot(attn.numpy(), label='Vector')
ax2.plot(attn_noised.numpy(), label='Vector with added noise')
ax2.tick_params(axis='both', which='both', labelsize=13)
ax2.set_xlabel('Audio frame index', fontsize=13)
ax2.set_ylabel('Activation', fontsize=13)
ax2.legend()

fig.tight_layout()


In [ ]:
durations = torch.cumsum(inp.out_duration, dim=0) * DURATION_SCALER / ms_per_step

target = pos_map[durations.long()]

showTensor(pos_feature[:100])

showTensor(target[:100])
showTensor(audio.cpu())

att = Attention(POS_DIM)
new_map, weights = att(target.unsqueeze(0), None, pos_feature.unsqueeze(0), None)
showTensor(weights.transpose(1, 2))
showTensor(label_vec.cpu())

## Models

In [ ]:
inp: Utterance = test_dataset[36]

audio = inp.features
length = audio.shape[0]

label_vec = inp.label_vec
actual_borders = inp.border
transcription = inp.in_transcription

labels_ids = np.argmax(transcription.cpu(), axis=1).tolist()
labels = [KNOWN_LABELS[lab] for lab in labels_ids]
labels_pos = actual_borders.cpu().tolist()

attention = position_model.with_weights(transcription.unsqueeze(0), None, audio.unsqueeze(0), None).detach().squeeze(0)

audio = audio.clone()
audio -= audio.min()
audio /= audio.max()

fig = plt.figure(figsize=(10, 9), constrained_layout=True)
gs = gridspec.GridSpec(ncols=2, nrows=4, height_ratios=[2, 6, 2, 3], width_ratios=[5, 2])

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[2, 0], sharex=ax1)
ax4 = fig.add_subplot(gs[1, 1], sharey=ax2)
ax5 = fig.add_subplot(gs[3, 0], sharex=ax1)
ax6 = fig.add_subplot(gs[3, 1], sharey=ax5)

ax1.imshow(audio.cpu().numpy().T, origin="lower", aspect='auto')
ax1.title.set_text("1. Audio features")
ax1.set_ylabel('Audio features', fontsize=13)
ax1.set_xlabel('Audio frames index', fontsize=13)

ax2.imshow(attention.cpu().numpy(), origin="lower", aspect='auto')
ax2.title.set_text("3. Attention matrix")
ax2.set_xlabel('Audio frames index', fontsize=13)
ax2.set_ylabel('Phoneme occurence order', fontsize=13)
# ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.set_yticks(range(len(labels)))
ax2.set_yticklabels([f"{i+1}. {p}" for i, p in enumerate(labels)])

ax3.plot(attention[3].cpu().numpy(), label='Attention score')
ax3.title.set_text("4. The activation of the third phoneme over the audio")
ax3.set_xlabel('Audio frames index', fontsize=13)
ax3.set_ylabel('Activation', fontsize=13)
ax3.legend()

ax4.imshow(transcription.cpu().numpy(), origin="lower", aspect='auto')
ax4.title.set_text("2. One-hot phoneme transcriptions")
ax4.set_ylabel('Phoneme occurence order', fontsize=13)
ax4.set_xlabel('One-hot phoneme ID', fontsize=13)

ax5.imshow((attention[3] * audio.T).cpu().numpy(), origin="lower", aspect='auto')
ax5.title.set_text("5. Audio weighted by the third phoneme attention")
ax5.set_xlabel('Audio frames index * attention weight', fontsize=13)
ax5.set_ylabel('Weighted features', fontsize=13)

ax6.imshow((attention[3] * audio.T).sum(1).unsqueeze(0).cpu().numpy().T, origin="lower", aspect=0.65)
# ax6.title.set_text("6. Vector of weighted sum of the audio features")
ax6.title.set_text("6. Weighted sum audio feature vector")
ax6.set_xticklabels([], minor=False)
ax6.set_yticklabels([], minor=False)
ax6.set_ylabel('Aduio features', fontsize=13)

fig.tight_layout()


In [ ]:
inp: Utterance = test_dataset[ 36  ]  # 36  1168 933

audio = inp.features.clone()
length = audio.shape[0]

label_vec = inp.label_vec
actual_borders = inp.border
transcription = inp.in_transcription
t_count, t_features = transcription.shape


# audio = (audio + audio * torch.rand_like(audio) * 3) / 3
attention = position_model.with_weights(transcription.unsqueeze(0), None, audio.unsqueeze(0), None).detach().squeeze(0)

audio = audio.clone()
audio -= audio.min()
audio /= audio.max()

labels_ids = np.argmax(transcription.cpu(), axis=1).tolist()
labels = [KNOWN_LABELS[lab] for lab in labels_ids]
labels_pos = actual_borders.cpu().tolist()

fig = plt.figure(figsize=(10, 9), constrained_layout=True)
gs = gridspec.GridSpec(ncols=1, nrows=3, height_ratios=[3, 10, 3])#, width_ratios=[5, 2])

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[2, 0], sharex=ax1)
# ax4 = fig.add_subplot(gs[1, 1], sharey=ax2)

ax1.imshow(audio.cpu().numpy().T, origin="lower", aspect='auto')
ax1.title.set_text("1. Audio features")
ax1.set_ylabel('Audio features', fontsize=13)
ax1.set_xlabel('Audio frames index', fontsize=13)


ax2.imshow(attention.cpu().numpy(), origin="lower", aspect='auto')
ax2.title.set_text("3. Attention matrix")
ax2.set_xlabel('Audio frames index', fontsize=13)
ax2.set_ylabel('Phoneme index', fontsize=13)
ax2.tick_params(axis='both', which='both', labelsize=13)
ax2.set_yticks(list( range(t_count)))
ax2.set_yticklabels([f"{i+1}. {p}{(i%2 == 6) * ' - - '}" for i, p in enumerate(labels)])

ax3.plot(attention[3].cpu().numpy(), label=f'4. {labels[3]}')
ax3.plot(attention[10 - 1].cpu().numpy(), label=f'10. {labels[9]}')
ax3.plot(attention[14 - 1].cpu().numpy(), label=f'14. {labels[13]}')
ax3.plot(attention[18 - 1].cpu().numpy(), label=f'18. {labels[17]}')
ax3.plot(attention[23 - 1].cpu().numpy(), label=f'23. {labels[22]}')
for border in actual_borders[[3, 9, 13, 17, 22]]:
    ax3.axvline(x=border)

ax3.title.set_text("4. The activation of four phonemes over the audio with actual borders")
ax3.set_xlabel('Audio frames index', fontsize=13)
ax3.set_ylabel('Activation', fontsize=13)
ax3.legend()

# ax4.imshow(transcription.cpu().numpy())
# ax4.title.set_text("2. One-hot phoneme transcriptions transposed")
# ax4.set_ylabel('Phoneme index', fontsize=13)
# ax4.set_xlabel('One-hot phoneme ID', fontsize=13)
fig.tight_layout()


fig = plt.figure(figsize=(10, 2), constrained_layout=True)
plt.plot(attention[3].cpu().numpy(), label=f'4. {labels[3]}')
plt.plot(attention[10 - 1].cpu().numpy(), label=f'10. {labels[9]}')
plt.plot(attention[14 - 1].cpu().numpy(), label=f'14. {labels[13]}')
plt.plot(attention[18 - 1].cpu().numpy(), label=f'18. {labels[17]}')
plt.plot(attention[23 - 1].cpu().numpy(), label=f'23. {labels[22]}')
for border in actual_borders[[3, 9, 13, 17, 22]]:
    plt.axvline(x=border)
plt.xlim([0, length-1])
plt.title("The activation of four phonemes over the audio with actual borders")
plt.xlabel('Audio frames index', fontsize=13)
plt.ylabel('Activation', fontsize=13)
plt.legend()
plt.plot()


index = 9
fig = plt.figure(figsize=(10, 4), constrained_layout=True)
att_local = attention[index].cpu().numpy()
predicted_border = (np.arange(length) * attention[index].cpu().numpy()).sum()

plt.plot(att_local, label=f'{index+1}. {labels[index]}', color=[ax2._get_patches_for_fill.get_next_color() for c in '123'][1])
# for border in actual_borders[[9]]:
#     plt.axvline(x=border, c='g')

s, e = 30, 65
for i, y in list(enumerate(att_local))[s:e:2]:
    plt.text(i-0.3, y, f'{y:.1f} * {i}', rotation=65)


plt.xlim([s, e])
plt.ylim([-0.05, 0.35])

plt.axvline(x=actual_borders[index]-0.15, c='g', label=f'Actual border ({actual_borders[index]:.1f})')
plt.axvline(x=predicted_border, c='b', label=f'Soft poitner border ({predicted_border:.1f})')
plt.axvline(x=50, c='r', label='Argmax border (50)')


plt.title("The activation value of phoneme /iy/ multiplied by the audio index value", fontsize=13)
plt.xlabel('Audio frames index', fontsize=13)
plt.ylabel('Activation', fontsize=13)
plt.legend()
# plt.text(21, 0.1, 'Actual attention', bbox={'facecolor': 'white', 'pad': 2})
plt.plot()




In [ ]:
inp: Utterance = test_dataset[36]

audio = inp.features.clone()
length = audio.shape[0]

label_vec = inp.label_vec
actual_borders = inp.border
transcription = inp.in_transcription
t_count, t_features = transcription.shape


# audio = (audio + audio * torch.rand_like(audio) * 3) / 3
attention = position_model.with_weights(transcription.unsqueeze(0), None, audio.unsqueeze(0), None).detach()
positions = position_model.weights_to_positions(attention).detach().squeeze(0).cpu().numpy()
attention = attention.squeeze(0)

audio = audio.clone()
audio -= audio.min()
audio /= audio.max()

labels_ids = np.argmax(transcription.cpu(), axis=1).tolist()[:-1]
labels = [KNOWN_LABELS[lab] for lab in labels_ids]
labels_pos = actual_borders.cpu().tolist()
label_colors = [ax2._get_patches_for_fill.get_next_color() for c in labels_ids]

fig = plt.figure(figsize=(10, 2), constrained_layout=True)
for i in range(0, len(labels_ids), 1):
    c = label_colors[i]
    plt.plot(attention[i].cpu().numpy(), label=f'{i+1}. {labels[i]}', c=c)
    plt.axvline(x=labels_pos[i], c=c, linestyle='-')
    plt.axvline(x=positions[i], c=c, linestyle='-.')

plt.xlim([10, 120])
plt.ylim([-0.05, 1.1])

plt.title("Predicted borders (dotted) compared with actual borders (solid)", fontsize=13)
plt.xlabel('Audio frames index', fontsize=13)
plt.ylabel('Activation', fontsize=13)
# plt.legend()
plt.plot()

In [ ]:
inp_batch: UtteranceBatch = next(b for b in train_dataset.batch(128))


position_model.eval()
attention = position_model.with_weights(
    inp_batch.in_transcription.padded, inp_batch.in_transcription.masks,
    inp_batch.features.padded,
    inp_batch.features.masks).detach().cpu().numpy()


batched_att = []
for i in range(64, 128, 8):
    ts, fs = (inp_batch.in_transcription.lengths[i], inp_batch.features.lengths[i])
    attention[i,ts:] = 0.25
    attention[i,:, fs:] = 0.25
    batched_att.append(attention[i])

    # plt.axvline(x=fs)
    # plt.axhline(y=ts)
    # plt.imshow(attention[i], origin="lower", aspect='auto')
    # plt.show()


fig = plt.figure(figsize=(8, 6), constrained_layout=True)
gs = gridspec.GridSpec(ncols=2, nrows=2)#, height_ratios=[3, 10, 3])#, width_ratios=[5, 2])

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[0, 1], sharex=ax1, sharey=ax1)
ax4 = fig.add_subplot(gs[1, 1], sharey=ax2, sharex=ax3)

index = -1
ax1.imshow(batched_att[index], origin="lower", aspect='auto')
ax1.title.set_text("1. Attention batch 1")
index = -2
ax2.imshow(batched_att[index], origin="lower", aspect='auto')
ax2.title.set_text("2. Attention batch 2")
index = -3
ax3.imshow(batched_att[index], origin="lower", aspect='auto')
ax3.title.set_text("3. Attention batch 3")
index = -4
ax4.imshow(batched_att[index], origin="lower", aspect='auto')
ax4.title.set_text("4. Attention batch 4")

fig.tight_layout()

In [ ]:
inp: Utterance = test_dataset[36]

audio = inp.features.clone()
length = audio.shape[0]

label_vec = inp.label_vec
actual_borders = inp.border
transcription = inp.in_transcription
t_count, t_features = transcription.shape

pos_encoding_out = position_model.with_position(transcription.unsqueeze(0), None, audio.unsqueeze(0), None).detach().cpu().squeeze(0)


fig = plt.figure(figsize=(10, 3), constrained_layout=True)
gs = gridspec.GridSpec(ncols=3, nrows=1)

labels_ids = np.argmax(transcription.cpu(), axis=1).tolist()
labels = [KNOWN_LABELS[lab] for lab in labels_ids]
labels_pos = actual_borders.cpu().tolist()

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharey=ax1)
ax3 = fig.add_subplot(gs[2], sharey=ax1)

ax1.imshow(inp.position.cpu().numpy().T, aspect='auto')
ax1.title.set_text("1. Position encoding actual")
ax1.set_xlabel('Phonemes ordered', fontsize=13)
ax1.set_ylabel('Position encoding features', fontsize=13)
ax1.tick_params(axis='both', which='both', labelsize=13)
ax1.set_xticks(list(range(t_count - 1))[::2])
ax1.set_xticklabels([f"{i+1}. {p}{(i%2 == 6) * ' - - '}" for i, p in enumerate(labels)][::2], rotation=45)

ax2.imshow(pos_encoding_out.cpu().numpy().T, aspect='auto')
ax2.title.set_text("2. Position encoding predicted")
ax2.set_xlabel('Phonemes ordered', fontsize=13)
ax2.tick_params(axis='both', which='both', labelsize=13)
ax2.set_xticks(list(range(t_count - 1))[::2])
ax2.set_xticklabels([f"{i+1}. {p}{(i%2 == 6) * ' - - '}" for i, p in enumerate(labels)][::2], rotation=45)

ax3.imshow((pos_encoding_out - inp.position).abs().cpu().numpy().T, aspect='auto')
ax3.title.set_text("3. Difference")
ax3.set_xlabel('Phonemes ordered', fontsize=13)
ax3.tick_params(axis='both', which='both', labelsize=13)
ax3.set_xticks(list(range(t_count - 1))[::2])
ax3.set_xticklabels([f"{i+1}. {p}{(i%2 == 6) * ' - - '}" for i, p in enumerate(labels)][::2], rotation=45)



fig.tight_layout()



In [ ]:
inp: Utterance = test_dataset[ 36  ]  # 36  1168 933

audio = inp.features.clone()
length = audio.shape[0]

label_vec = inp.label_vec
actual_borders = inp.border
transcription = inp.in_transcription
t_count, t_features = transcription.shape

n = 15
audio[40 + n:50 + n] = audio.min()

# audio = (audio + audio * torch.rand_like(audio) * 3) / 3
attention = position_model.with_weights(transcription.unsqueeze(0), None, audio.unsqueeze(0), None).detach().squeeze(0)

audio = audio.clone()
audio -= audio.min()
audio /= audio.max()

labels_ids = np.argmax(transcription.cpu(), axis=1).tolist()
labels = [KNOWN_LABELS[lab] for lab in labels_ids]
labels_pos = actual_borders.cpu().tolist()

fig = plt.figure(figsize=(10, 9), constrained_layout=True)
gs = gridspec.GridSpec(ncols=1, nrows=3, height_ratios=[3, 10, 3])#, width_ratios=[5, 2])

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[2, 0], sharex=ax1)
# ax4 = fig.add_subplot(gs[1, 1], sharey=ax2)

ax1.imshow(audio.cpu().numpy().T, origin="lower", aspect='auto')
ax1.title.set_text("1. Audio features")
ax1.set_ylabel('Audio features', fontsize=13)
ax1.set_xlabel('Audio frames index', fontsize=13)


ax2.imshow(attention.cpu().numpy(), origin="lower", aspect='auto')
ax2.title.set_text("3. Attention matrix")
ax2.set_xlabel('Audio frames index', fontsize=13)
ax2.set_ylabel('Phoneme index', fontsize=13)
ax2.tick_params(axis='both', which='both', labelsize=13)
ax2.set_yticks(list( range(t_count)))
ax2.set_yticklabels([f"{i+1}. {p}{(i%2 == 6) * ' - - '}" for i, p in enumerate(labels)])

ax3.plot(attention[3].cpu().numpy(), label=f'4. {labels[3]}')
ax3.plot(attention[10 - 1].cpu().numpy(), label=f'10. {labels[9]}')
ax3.plot(attention[14 - 1].cpu().numpy(), label=f'14. {labels[13]}')
ax3.plot(attention[18 - 1].cpu().numpy(), label=f'18. {labels[17]}')
ax3.plot(attention[23 - 1].cpu().numpy(), label=f'23. {labels[22]}')
for border in actual_borders[[3, 9, 13, 17, 22]]:
    ax3.axvline(x=border)

ax3.title.set_text("4. The activation of five phonemes over the audio with actual borders")
ax3.set_xlabel('Audio frames index', fontsize=13)
ax3.set_ylabel('Activation', fontsize=13)
ax3.legend()

# ax4.imshow(transcription.cpu().numpy())
# ax4.title.set_text("2. One-hot phoneme transcriptions transposed")
# ax4.set_ylabel('Phoneme index', fontsize=13)
# ax4.set_xlabel('One-hot phoneme ID', fontsize=13)
fig.tight_layout()




In [ ]:
inp: Utterance = test_dataset[ 36  ]  # 36  1168 933

audio = inp.features.clone()
length = audio.shape[0]

label_vec = inp.label_vec
actual_borders = inp.border
transcription = inp.in_transcription
t_count, t_features = transcription.shape

# audio = (audio + audio * torch.rand_like(audio) * 3) / 3
attention = position_model.with_weights(transcription.unsqueeze(0), None, audio.unsqueeze(0), None).detach().squeeze(0)

audio = audio.clone()
audio -= audio.min()
audio /= audio.max()

labels_ids = np.argmax(transcription.cpu(), axis=1).tolist()
labels = [KNOWN_LABELS[lab] for lab in labels_ids]
labels_pos = actual_borders.cpu().tolist()

fig = plt.figure(figsize=(10, 6), constrained_layout=True)
gs = gridspec.GridSpec(ncols=1, nrows=2, height_ratios=[4, 5])#, width_ratios=[5, 2])

ax2 = fig.add_subplot(gs[1])
ax1 = fig.add_subplot(gs[0], sharex=ax2)


audio = (audio + audio * torch.rand_like(audio) * 3) / 3
a = audio.cpu().numpy()
a[:,13:] *= np.cumsum(vec).reshape(-1, 1)

ax2.imshow(a.T, origin="lower", aspect='auto')
ax2.title.set_text("2. Audio encoder features")
ax2.set_ylabel('Audio encoder features', fontsize=13)
ax2.set_xlabel('Audio frames index', fontsize=13)


vec = attention[9].cpu().numpy()
print(vec.shape)
ax1.plot(vec, label=f'9. Attention score {labels[9]}')
ax1.plot(np.cumsum(vec), label=f'9. Cumulative sum {labels[9]}')
ax1.title.set_text(f"1. The attention score and cumulative sum for phoneme {labels[9]}")
ax1.set_xlabel('Audio frames index', fontsize=13)
ax1.set_ylabel('Activation', fontsize=13)
ax1.legend()

fig.tight_layout()


## Duration

In [ ]:
inp: Utterance = test_dataset[36]

audio = inp.features
length = audio.shape[0]
label_vec = inp.label_vec
actual_borders = inp.border
transcription = inp.in_transcription



        # f, axarr = plt.subplots(1, 2, figsize=(13, 3))
        # axarr[0].bar(range(len(bins)-1), np.round(hist).astype(int))
        # axarr[0].set_xticklabels([f"{x:.0f}{unit}" for x in bins], fontdict=None, minor=False)
        # axarr[0].yaxis.set_major_formatter(FormatStrFormatter('%d%%'))
        # axarr[1].hist(errors, bins=15)
        # axarr[1].xaxis.set_major_formatter(FormatStrFormatter(f'%d{unit}'))


# 10ms b-64  97
ms_10 = [
    [5, 47.4],
    [10, 76.7],
    [15, 88.1],
    [20, 93.0],
]

# 5ms  b 16  250.6
ms_5 = [
    [5, 48.6],
    [10, 76.9],
    [15, 88.3],
    [20, 93.0],
]

# 15ms b 128 = 26.2
ms_15 = [
    [5, 34.9],
    [10, 66.4],
    [15, 83.8],
    [20, 91.1],
]

def autolabel(rects, ax):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 0),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')




fig = plt.figure(figsize=(9, 4), constrained_layout=True)
gs = gridspec.GridSpec(ncols=2, nrows=1, width_ratios=[18, 8])

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])


x = np.arange(4) * 1.5
width = 0.4  # the width of the bars
rects1 = ax1.bar(x + width * 1, [e[1] for e in ms_5], width, label='5ms step, 16 batch')
rects2 = ax1.bar(x + width * 0, [e[1] for e in ms_10], width, label='10ms step, 64 batch')
rects3 = ax1.bar(x - width * 1, [e[1] for e in ms_15], width, label='15ms step, 128 batch')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax1.set_ylabel('Percentage')
ax1.set_xlabel('Agreement boundary')
ax1.set_title('1. Agreement boundary improvements')
ax1.set_xticks(x)
ax1.set_xticklabels([f"{e[0]}ms boundary" for e in ms_15])
ax1.yaxis.set_major_formatter(FormatStrFormatter('%d%%'))
ax1.legend(loc="lower right")
autolabel(rects1, ax1)
autolabel(rects2, ax1)
autolabel(rects3, ax1)

print()

rects1 = ax2.bar([5, 10, 15], [250.6 / 2, 97.1 / 2, 26.2 / 2], 3, color=[ax2._get_patches_for_fill.get_next_color() for c in '123'])
# Add some text for labels, title and custom x-axis tick labels, etc.
ax2.set_ylabel('Training time in seconds')
ax2.set_xlabel('Audio frame step size')
ax2.set_title('2. Training time improvements')
ax2.set_xticks([5, 10, 15])
ax2.set_xticklabels([f"{e}ms size" for e in [5, 10, 15]])
ax2.yaxis.set_major_formatter(FormatStrFormatter('%dsec'))
autolabel(rects1, ax2)

fig.tight_layout()
